# game EIU

# 1 libraries

In [8]:
# pip install -U pandasql

In [322]:
from datavolley import read_dv
import pandas as pd
from datetime import datetime
import sqlite3
from IPython.display import Markdown
import pygwalker as pyg
import streamlit as st
# from pandasql import sqldf
import sqlalchemy as sa
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
# from kaleido.scopes.plotly import PlotlyScope
from dash import Dash, html, dcc, callback, Output, Input
import chart_studio
# import dataframe_image as dfi
username = 'reies'
api_key = 'pgusldsavElAxIebHrHB'

chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

# 2 database

In [323]:
conn = sqlite3.connect('eiu2024.db')

In [324]:
cn = conn.cursor()

# 3 open datavolley

In [939]:
dv_instance = read_dv.DataVolley("dv_files\&20240914-7-Eas-Bra.dvw") # Replace `None` with path of your dvw file 
df = dv_instance.get_plays()
df

<>:1: SyntaxWarning:

invalid escape sequence '\&'

<>:1: SyntaxWarning:

invalid escape sequence '\&'

C:\Users\456402\AppData\Local\Temp\ipykernel_7612\2270288852.py:1: SyntaxWarning:

invalid escape sequence '\&'



,match_id,video_file_number,video_time,code,team,player_number,player_name,player_id,skill,evaluation_code,...,set_number,home_team,visiting_team,home_team_id,visiting_team_id,point_won_by,serving_team,receiving_team,rally_number,possesion_number
0,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,1,,*P23>LUp,Eastern Illinois University,NaN,NaN,NaN,NaN,NaN,...,1,Eastern Illinois University,Bradley University,232,202,NaN,NaN,NaN,0,0
1,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,1,,*z1>LUp,Eastern Illinois University,NaN,NaN,NaN,NaN,NaN,...,1,Eastern Illinois University,Bradley University,232,202,NaN,NaN,NaN,0,1
2,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,1,,aP12>LUp,Bradley University,NaN,NaN,NaN,NaN,NaN,...,1,Eastern Illinois University,Bradley University,232,202,NaN,NaN,NaN,0,1
3,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,1,,az1>LUp,Bradley University,NaN,NaN,NaN,NaN,NaN,...,1,Eastern Illinois University,Bradley University,232,202,NaN,NaN,NaN,0,1
4,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,1,664,*23SM-~~~16C,Eastern Illinois University,23,Sylvia Hasz,HAS-SYL,Serve,-,...,1,Eastern Illinois University,Bradley University,232,202,Bradley University,Eastern Illinois University,Bradley University,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1688,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,1,9207,*23ET!K1F~3D,Eastern Illinois University,23,Sylvia Hasz,HAS-SYL,Set,!,...,5,Eastern Illinois University,Bradley University,232,202,Bradley University,Eastern Illinois University,Bradley University,27,2
1689,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,1,9208,*10AT=X5~46DH2~6H2,Eastern Illinois University,10,Matilda Dahlstrom,DAL-MAT,Attack,=,...,5,Eastern Illinois University,Bradley University,232,202,Bradley University,Eastern Illinois University,Bradley University,27,2
1690,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,1,9208,a$$&H#,Bradley University,NaN,NaN,NaN,NaN,#,...,5,Eastern Illinois University,Bradley University,232,202,Bradley University,Eastern Illinois University,Bradley University,27,3
1691,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,1,9218,ap12:15,Bradley University,NaN,NaN,NaN,Point,NaN,...,5,Eastern Illinois University,Bradley University,232,202,Bradley University,Eastern Illinois University,Bradley University,27,3


In [485]:
# df.info()

# 4 clean & transform data

In [940]:
# drop columns
df.drop(['video_file_number', 'video_time', 'code', 'custom_code'], axis=1, inplace=True)

In [941]:
# delete NULL in Skill column
df = df[df['skill'].notna()]

In [942]:
# drop Point value from Skill column
df = df[df['skill'] != 'Point']


In [943]:
# add unique id to each row
df['point_id'] = range(1, len(df)+1)

In [944]:
# HOME switch from rotations to Rows
df.loc[df['home_setter_position'] == '1', 'home_setter_row'] = '1'
df.loc[df['home_setter_position'] == '6', 'home_setter_row'] = '2'
df.loc[df['home_setter_position'] == '5', 'home_setter_row'] = '3'
df.loc[df['home_setter_position'] == '4', 'home_setter_row'] = '4'
df.loc[df['home_setter_position'] == '3', 'home_setter_row'] = '5'
df.loc[df['home_setter_position'] == '2', 'home_setter_row'] = '6'

In [945]:
# ALL_Setter switch from rotations to Rows
df.loc[df['setter_position'] == '1', 'setter_row'] = '1'
df.loc[df['setter_position'] == '6', 'setter_row'] = '2'
df.loc[df['setter_position'] == '5', 'setter_row'] = '3'
df.loc[df['setter_position'] == '4', 'setter_row'] = '4'
df.loc[df['setter_position'] == '3', 'setter_row'] = '5'
df.loc[df['setter_position'] == '2', 'setter_row'] = '6'

In [946]:
# VISIT switch from rotations to Rows
df.loc[df['visiting_setter_position'] == '1', 'visiting_setter_row'] = '1'
df.loc[df['visiting_setter_position'] == '6', 'visiting_setter_row'] = '2'
df.loc[df['visiting_setter_position'] == '5', 'visiting_setter_row'] = '3'
df.loc[df['visiting_setter_position'] == '4', 'visiting_setter_row'] = '4'
df.loc[df['visiting_setter_position'] == '3', 'visiting_setter_row'] = '5'
df.loc[df['visiting_setter_position'] == '2', 'visiting_setter_row'] = '6'

In [947]:
# home rename setters positions
df.rename(columns={"home_p1": "home_p1","home_p6": "home_p2", "home_p5": "home_p3", "home_p4": "home_p4", "home_p3": "home_p5", "home_p2": "home_p6"}, inplace=True)

In [948]:
# visit rename setters positions
df.rename(columns={"visiting_p1": "visiting_p1","visiting_p6": "visiting_p2", "visiting_p5": "visiting_p3", "visiting_p4": "visiting_p4", "visiting_p3": "visiting_p5", "visiting_p2": "visiting_p6"}, inplace=True)

In [949]:
# transform columns string to integer
df[['set_number','player_number', 'setter_position', 'home_p1', 'home_p2', 'home_p3', 'home_p4', 'home_p5', 'home_p6', 'visiting_p1', 'visiting_p2', 'visiting_p3', 'visiting_p4', 'visiting_p5', 'visiting_p6', 'home_setter_position', 'visiting_setter_position']] = df[['set_number','player_number', 'setter_position', 'home_p1', 'home_p2', 'home_p3', 'home_p4', 'home_p5', 'home_p6', 'visiting_p1', 'visiting_p2', 'visiting_p3', 'visiting_p4', 'visiting_p5', 'visiting_p6', 'home_setter_position', 'visiting_setter_position']].astype(int)

In [950]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1242 entries, 4 to 1689
Data columns (total 56 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   match_id                  1242 non-null   object
 1   team                      1242 non-null   object
 2   player_number             1242 non-null   int32 
 3   player_name               1240 non-null   object
 4   player_id                 1240 non-null   object
 5   skill                     1242 non-null   object
 6   evaluation_code           1242 non-null   object
 7   setter_position           1242 non-null   int32 
 8   attack_code               298 non-null    object
 9   set_code                  201 non-null    object
 10  set_type                  251 non-null    object
 11  start_zone                853 non-null    object
 12  end_zone                  1153 non-null   object
 13  end_subzone               1037 non-null   object
 14  num_players_numeric     

## CAREFUL... DOUBLE CHECK THE INFO BEFORE

In [951]:
# team names
df['team'].unique()

array(['Eastern Illinois University', 'Bradley University'], dtype=object)

In [952]:
df.fillna(0, inplace=True)

In [953]:
# HOMEreplacing team`s names
df.replace('Eastern Illinois University','EIU', inplace=True)

In [954]:
# VISIT replacing team`s names
df.replace('Bradley University','Bradley', inplace=True)

In [960]:
# adding columns: year, game number, 'non conference' or 'conference'
df['year'] =  2024
df['game_number'] =  7
df['season'] = 'non-conference'
df['date'] = datetime(2024, 9, 14).date()

In [961]:
# home and visiting teams code
h_id = df['home_team_id'].iloc[0]
v_id = df['visiting_team_id'].iloc[0]
g_num = df['game_number'].iloc[0]
season = df['season'].iloc[0]
season_year = df['year'].iloc[0]

# home and visiting team names
h_name = df['home_team'].iloc[0]
v_name = df['visiting_team'].iloc[0]
print(v_name + '`s code: ' + v_id + "\n" )
print(h_name + '`s code: ' + h_id )

home = h_name
visiting = v_name
# skill
# skill = df['skill'].iloc[0]

Bradley`s code: 202

EIU`s code: 232


In [962]:
# Add color for 'Marquette University' and 'St. John`s University'
color_map = {h_name: 'blue', v_name: 'cyan'}
df['color'] = df['point_won_by'].map(color_map)

In [963]:
df

,match_id,team,player_number,player_name,player_id,skill,evaluation_code,setter_position,attack_code,set_code,...,possesion_number,point_id,home_setter_row,setter_row,visiting_setter_row,year,game_number,season,date,color
4,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,23,Sylvia Hasz,HAS-SYL,Serve,-,1,0,0,...,0,1,1,1,1,2024,7,non-conference,2024-09-14,cyan
5,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,8,Irene Mostardini,-480052,Reception,+,1,0,0,...,1,2,1,1,1,2024,7,non-conference,2024-09-14,cyan
6,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,12,Maria Drapp,-515752,Set,+,1,0,K7,...,1,3,1,1,1,2024,7,non-conference,2024-09-14,cyan
7,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,11,Iva Popovic,-515751,Attack,#,1,X6,0,...,1,4,1,1,1,2024,7,non-conference,2024-09-14,cyan
8,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,17,Julia Stanev,STA-JUL,Block,=,1,0,0,...,2,5,1,1,1,2024,7,non-conference,2024-09-14,cyan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1685,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,12,Maria Drapp,-515752,Set,+,1,0,K1,...,1,1238,1,1,1,2024,7,non-conference,2024-09-14,cyan
1686,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,25,Dakota Elliott,-515755,Attack,-,1,C1,0,...,1,1239,1,1,1,2024,7,non-conference,2024-09-14,cyan
1687,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,5,Ariadna Pereles,PER-ARI,Dig,+,1,0,0,...,2,1240,1,1,1,2024,7,non-conference,2024-09-14,cyan
1688,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,23,Sylvia Hasz,HAS-SYL,Set,!,1,0,K1,...,2,1241,1,1,1,2024,7,non-conference,2024-09-14,cyan


## CAREFULLLLL Table name .... insert df to database

In [964]:
# inserting DF to SQLite: if_exists = 'replace' or if_exists = 'append'
df.to_sql('eiu24_game',
            conn,
         if_exists = 'append')

1242

In [965]:
conn.commit()

In [966]:
# cn.close()
# conn.close()

In [967]:
pd.read_sql_query('select * from eiu24_game', conn)

,index,match_id,team,player_number,player_name,player_id,skill,evaluation_code,setter_position,attack_code,...,possesion_number,point_id,home_setter_row,setter_row,visiting_setter_row,year,game_number,season,date,color
0,4,39fab150-da68-4a49-a94f-392c82e77bec,Marquette,12,Carsen Murray,-317104,Serve,#,5,0,...,0,1,1,3,3,2024,1,non-conference,2024-08-30,cyan
1,5,39fab150-da68-4a49-a94f-392c82e77bec,EIU,5,Ariadna Pereles,-283855,Reception,=,1,0,...,1,2,1,1,3,2024,1,non-conference,2024-08-30,cyan
2,11,39fab150-da68-4a49-a94f-392c82e77bec,Marquette,2,Molly Berezowitz,-438810,Serve,-,5,0,...,0,3,1,3,3,2024,1,non-conference,2024-08-30,cyan
3,12,39fab150-da68-4a49-a94f-392c82e77bec,EIU,5,Ariadna Pereles,-283855,Reception,+,1,0,...,1,4,1,1,3,2024,1,non-conference,2024-08-30,cyan
4,13,39fab150-da68-4a49-a94f-392c82e77bec,EIU,9,Catalina Rochaix,-371177,Set,#,1,0,...,1,5,1,1,3,2024,1,non-conference,2024-08-30,cyan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6885,1685,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,12,Maria Drapp,-515752,Set,+,1,0,...,1,1238,1,1,1,2024,7,non-conference,2024-09-14,cyan
6886,1686,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,25,Dakota Elliott,-515755,Attack,-,1,C1,...,1,1239,1,1,1,2024,7,non-conference,2024-09-14,cyan
6887,1687,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,5,Ariadna Pereles,PER-ARI,Dig,+,1,0,...,2,1240,1,1,1,2024,7,non-conference,2024-09-14,cyan
6888,1688,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,23,Sylvia Hasz,HAS-SYL,Set,!,1,0,...,2,1241,1,1,1,2024,7,non-conference,2024-09-14,cyan


# 5 Dummies

In [968]:
# calling G Walker-Tableau
# pyg.walk(df)


In [969]:
df_dumm = pd.get_dummies(df, columns= ['serving_team', 'evaluation_code', 'point_won_by'], dtype= int )
df_dumm

,match_id,team,player_number,player_name,player_id,skill,setter_position,attack_code,set_code,set_type,...,serving_team_Bradley,serving_team_EIU,evaluation_code_!,evaluation_code_#,evaluation_code_+,evaluation_code_-,evaluation_code_/,evaluation_code_=,point_won_by_Bradley,point_won_by_EIU
4,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,23,Sylvia Hasz,HAS-SYL,Serve,1,0,0,0,...,0,1,0,0,0,1,0,0,1,0
5,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,8,Irene Mostardini,-480052,Reception,1,0,0,0,...,0,1,0,0,1,0,0,0,1,0
6,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,12,Maria Drapp,-515752,Set,1,0,K7,B,...,0,1,0,0,1,0,0,0,1,0
7,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,11,Iva Popovic,-515751,Attack,1,X6,0,0,...,0,1,0,1,0,0,0,0,1,0
8,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,17,Julia Stanev,STA-JUL,Block,1,0,0,0,...,0,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1685,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,12,Maria Drapp,-515752,Set,1,0,K1,C,...,0,1,0,0,1,0,0,0,1,0
1686,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,25,Dakota Elliott,-515755,Attack,1,C1,0,0,...,0,1,0,0,0,1,0,0,1,0
1687,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,5,Ariadna Pereles,PER-ARI,Dig,1,0,0,0,...,0,1,0,0,1,0,0,0,1,0
1688,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,23,Sylvia Hasz,HAS-SYL,Set,1,0,K1,F,...,0,1,1,0,0,0,0,0,1,0


In [970]:
df_dumm.rename(columns = {'evaluation_code_#' : 'perfect', 'evaluation_code_+' : 'positive', 'evaluation_code_!' : 'medium', 'evaluation_code_-' : 'negative','evaluation_code_/' : 'slash','evaluation_code_=' : 'error'}, inplace=True)

In [971]:
df_dumm.head()

,match_id,team,player_number,player_name,player_id,skill,setter_position,attack_code,set_code,set_type,...,serving_team_Bradley,serving_team_EIU,medium,perfect,positive,negative,slash,error,point_won_by_Bradley,point_won_by_EIU
4,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,23,Sylvia Hasz,HAS-SYL,Serve,1,0,0,0,...,0,1,0,0,0,1,0,0,1,0
5,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,8,Irene Mostardini,-480052,Reception,1,0,0,0,...,0,1,0,0,1,0,0,0,1,0
6,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,12,Maria Drapp,-515752,Set,1,0,K7,B,...,0,1,0,0,1,0,0,0,1,0
7,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,11,Iva Popovic,-515751,Attack,1,X6,0,0,...,0,1,0,1,0,0,0,0,1,0
8,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,17,Julia Stanev,STA-JUL,Block,1,0,0,0,...,0,1,0,0,0,0,0,1,1,0


## CAREFULLLL... writing DUMMIES to eiu_dummy

In [972]:
# inserting DF to SQLite: if_exists = 'replace' or if_exists = 'append'
# df_dumm.to_sql('eiu24_win_dummy',
#             conn,
#          if_exists = 'append')

In [973]:
# conn.commit()

In [974]:
pd.read_sql_query('select * from eiu24_win_dummy', conn)

,index,match_id,team,player_number,player_name,player_id,skill,setter_position,attack_code,set_code,...,serving_team_EIU,serving_team_SDakota,medium,perfect,positive,negative,slash,error,point_won_by_EIU,point_won_by_SDakota
0,4,0496a950-3798-409c-8f0e-d9e415a90b51,EIU,9,Catalina Rochaix,ROC-CAT,Serve,1,0,0,...,1,0,1,0,0,0,0,0,0,1
1,5,0496a950-3798-409c-8f0e-d9e415a90b51,SDakota,16,Katie Van Egdom,-385232,Reception,2,0,0,...,1,0,1,0,0,0,0,0,0,1
2,6,0496a950-3798-409c-8f0e-d9e415a90b51,SDakota,16,Katie Van Egdom,-385232,Attack,2,XP,0,...,1,0,0,0,0,1,0,0,0,1
3,7,0496a950-3798-409c-8f0e-d9e415a90b51,EIU,9,Catalina Rochaix,ROC-CAT,Dig,1,0,0,...,1,0,0,0,1,0,0,0,0,1
4,8,0496a950-3798-409c-8f0e-d9e415a90b51,EIU,2,Laura Paniagua,LAU-PAN,Set,1,0,0,...,1,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,1084,0496a950-3798-409c-8f0e-d9e415a90b51,EIU,21,Ireland Hieb,IRL-HIE,Block,5,0,0,...,1,0,0,1,0,0,0,0,1,0
794,1087,0496a950-3798-409c-8f0e-d9e415a90b51,EIU,20,Julia Udell,UDE-JUL,Serve,5,0,0,...,1,0,0,0,0,1,0,0,0,1
795,1088,0496a950-3798-409c-8f0e-d9e415a90b51,SDakota,16,Katie Van Egdom,-385232,Reception,6,0,0,...,1,0,0,1,0,0,0,0,0,1
796,1089,0496a950-3798-409c-8f0e-d9e415a90b51,SDakota,5,Rylee Martin,-434390,Set,6,0,K1,...,1,0,0,0,1,0,0,0,0,1


# 7 Set Played, set played calculation & winner team

In [975]:
# total set played
set = df['set_number'].iloc[-1]
set

5

In [976]:
# defining the team winner and loser
def winner(set):
    if set == 3 and (df['home_team_score'].iloc[-1] > df['visiting_team_score'].iloc[-1]):
        return(f"{h_name} win 3-0 vs {v_name}")
    elif set == 3 and (df['visiting_team_score'].iloc[-1] > df['home_team_score'].iloc[-1]):
        return(f"{v_name} win 3-0 vs {h_name}")
    elif set == 4 and (df['home_team_score'].iloc[-1] > df['visiting_team_score'].iloc[-1]):
        return(f"{h_name} win 3-1 vs {v_name}")
    elif set == 4 and (df['visiting_team_score'].iloc[-1] > df['home_team_score'].iloc[-1]):
        return(f"{v_name} win 3-1 vs {h_name}")
    elif set == 5 and (df['home_team_score'].iloc[-1] > df['visiting_team_score'].iloc[-1]):
        return(f"{h_name} win 3-2 vs {v_name}")
    else:
        return(f"{v_name} win 3-2 vs {h_name}")

win_team = winner(set)

win_team

'Bradley win 3-2 vs EIU'

# 8 Serving Rallies

In [977]:
# total serving rallies
df_serve = df[df['skill']== 'Serve']
df_serve

,match_id,team,player_number,player_name,player_id,skill,evaluation_code,setter_position,attack_code,set_code,...,possesion_number,point_id,home_setter_row,setter_row,visiting_setter_row,year,game_number,season,date,color
4,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,23,Sylvia Hasz,HAS-SYL,Serve,-,1,0,0,...,0,1,1,1,1,2024,7,non-conference,2024-09-14,cyan
11,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,11,Iva Popovic,-515751,Serve,-,6,0,0,...,0,6,1,2,2,2024,7,non-conference,2024-09-14,blue
18,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,10,Matilda Dahlstrom,DAL-MAT,Serve,!,6,0,0,...,0,11,2,2,2,2024,7,non-conference,2024-09-14,blue
27,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,10,Matilda Dahlstrom,DAL-MAT,Serve,+,6,0,0,...,0,19,2,2,2,2024,7,non-conference,2024-09-14,cyan
34,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,8,Irene Mostardini,-480052,Serve,+,5,0,0,...,0,23,2,3,3,2024,7,non-conference,2024-09-14,cyan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,12,Maria Drapp,-515752,Serve,-,1,0,0,...,0,1193,6,1,1,2024,7,non-conference,2024-09-14,cyan
1641,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,12,Maria Drapp,-515752,Serve,+,1,0,0,...,0,1203,6,1,1,2024,7,non-conference,2024-09-14,cyan
1650,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,12,Maria Drapp,-515752,Serve,+,1,0,0,...,0,1210,6,1,1,2024,7,non-conference,2024-09-14,cyan
1659,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,12,Maria Drapp,-515752,Serve,+,1,0,0,...,0,1216,6,1,1,2024,7,non-conference,2024-09-14,blue


In [978]:
# home team serving rallies
df_serve_h = df_serve[df_serve['serving_team']== h_name]
# df_serve_h

In [979]:
# visiting team serving rallies
df_serve_v = df_serve[df_serve['serving_team']== v_name]
# df_serve_v

In [980]:
# DUMMIES ALL SERVE sequence
df_serve_dumm = pd.get_dummies(df_serve, columns= ['serving_team', 'evaluation_code', 'point_won_by'], dtype= int )
# df_serve_dumm.info()

In [981]:
# ALL Serve sequence
serve_seq = df_serve_dumm[['year','season','date','game_number','set_number', 'home_team_score', 'visiting_team_score', 'rally_number',  f'serving_team_{v_name}', f'serving_team_{h_name}']]
# serve_seq

In [982]:
# HOME Serve sequence by ROW
serve_seq_h = df_serve_dumm[['year','season','date','game_number','set_number','rally_number', 'home_team_score', 'visiting_team_score', 'home_setter_row', f'serving_team_{h_name}' ]]
serve_seq_h

,year,season,date,game_number,set_number,rally_number,home_team_score,visiting_team_score,home_setter_row,serving_team_EIU
4,2024,non-conference,2024-09-14,7,1,1,0,1,1,1
11,2024,non-conference,2024-09-14,7,1,2,1,1,1,0
18,2024,non-conference,2024-09-14,7,1,3,2,1,2,1
27,2024,non-conference,2024-09-14,7,1,4,2,2,2,1
34,2024,non-conference,2024-09-14,7,1,5,2,3,2,0
...,...,...,...,...,...,...,...,...,...,...
1629,2024,non-conference,2024-09-14,7,5,23,11,12,6,0
1641,2024,non-conference,2024-09-14,7,5,24,11,13,6,0
1650,2024,non-conference,2024-09-14,7,5,25,11,14,6,0
1659,2024,non-conference,2024-09-14,7,5,26,12,14,6,0


In [983]:
# VISIT Serve sequence by ROW
serve_seq_v = df_serve_dumm[['set_number','rally_number', 'visiting_setter_row', f'serving_team_{v_name}']]
# serve_seq_v.info()

In [984]:
# HOME Serve sequence by ROW-Set
serve_seq_h_s1 = serve_seq_h[serve_seq_h['set_number'] == 1][['set_number', 'rally_number', 'home_setter_row', f'serving_team_{h_name}']]
serve_seq_h_s2 = serve_seq_h[serve_seq_h['set_number'] == 2][['set_number', 'rally_number', 'home_setter_row', f'serving_team_{h_name}']]
serve_seq_h_s3 = serve_seq_h[serve_seq_h['set_number'] == 3][['set_number', 'rally_number', 'home_setter_row', f'serving_team_{h_name}']]
serve_seq_h_s4 = serve_seq_h[serve_seq_h['set_number'] == 4][['set_number', 'rally_number', 'home_setter_row', f'serving_team_{h_name}']]
serve_seq_h_s5 = serve_seq_h[serve_seq_h['set_number'] == 5][['set_number', 'rally_number', 'home_setter_row', f'serving_team_{h_name}']]
# serve_seq_h_s1

In [985]:
# VISIT Serve sequence by ROW-Set
serve_seq_v_s1 = serve_seq_v[serve_seq_v['set_number'] == 1][['set_number', 'rally_number', 'visiting_setter_row', f'serving_team_{v_name}']]
serve_seq_v_s2 = serve_seq_v[serve_seq_v['set_number'] == 2][['set_number', 'rally_number', 'visiting_setter_row', f'serving_team_{v_name}']]
serve_seq_v_s3 = serve_seq_v[serve_seq_v['set_number'] == 3][['set_number', 'rally_number', 'visiting_setter_row', f'serving_team_{v_name}']]
serve_seq_v_s4 = serve_seq_v[serve_seq_v['set_number'] == 4][['set_number', 'rally_number', 'visiting_setter_row', f'serving_team_{v_name}']]
serve_seq_v_s5 = serve_seq_v[serve_seq_v['set_number'] == 5][['set_number', 'rally_number', 'visiting_setter_row', f'serving_team_{v_name}']]
# serve_seq_v1

## pivot home serve 

In [986]:
# HOME PIVOT Serve-pass sequence
pivot_serve_seq_h = serve_seq_h.pivot_table(
    values=[f'serving_team_{h_name}'], 
    index=['rally_number'], 
    columns=['set_number', 'home_setter_row']
).fillna('')
print(f"{h_name} Serve vs Receive sequence by Row")
pivot_serve_seq_h

EIU Serve vs Receive sequence by Row


serving_team_EIU                                               \
set_number                     1                             2                  
home_setter_row                1    2    3    4    5    6    1    2    3    4   
rally_number                                                                    
1                            1.0                           0.0                  
2                            0.0                                1.0             
3                                 1.0                           1.0             
4                                 1.0                           1.0             
5                                 0.0                           0.0             
6                                 0.0                                1.0        
7                                 0.0                                0.0        
8                                 0.0                                0.0        
9                                 0.0                                     1.0   
10                                0.0                                     0.0   
11                                0.0                                           
12                                0.0                                           
13                                     1.0                                      
14                                     0.0                                      
15                                     0.0                                      
16                                     0.0                                      
17                                     1.0                                      
18                                     0.0                                      
19                                          1.0                                 
20                                          0.0            1.0                  
21                                          0.0            1.0                  
22                                               1.0       1.0                  
23                                                    1.0  0.0                  
24                                                    1.0       1.0             
25                                                    1.0       1.0             
26                                                    0.0       0.0             
27                                                    0.0       0.0             
28                           1.0                                     1.0        
29                           1.0                                     1.0        
30                           0.0                                     0.0        
31                                1.0                                1.0        
32                                1.0                                0.0        
33                                1.0                                0.0        
34                                0.0                                     1.0   
35                                     1.0                                1.0   
36                                     1.0                                1.0   
37                                     0.0                                0.0   
38                                     0.0                                      
39                                          1.0                                 
40                                          0.0                                 
41                                          0.0                                 
42                                          0.0            1.0                  
43                                                         1.0                  
44                                                         0.0                  
45                                                              1.0             

                 ...                          

## pivot visit serve 

In [987]:
# VISIT PIVOT Serve-pass sequence
pivot_serve_seq_v = serve_seq_v.pivot_table(
    values=f'serving_team_{v_name}', 
    index=['rally_number'], 
    columns=['set_number', 'visiting_setter_row']
).fillna('')
print(f"{v_name} Serve vs Receive sequence by Row")
pivot_serve_seq_v

Bradley Serve vs Receive sequence by Row


set_number             1                             2                 ...  \
visiting_setter_row    1    2    3    4    5    6    1    2    3    4  ...   
rally_number                                                           ...   
1                    0.0                           1.0                 ...   
2                         1.0                      0.0                 ...   
3                         0.0                      0.0                 ...   
4                         0.0                      0.0                 ...   
5                              1.0                      1.0            ...   
6                              1.0                      0.0            ...   
7                              1.0                           1.0       ...   
8                              1.0                           1.0       ...   
9                              1.0                           0.0       ...   
10                             1.0                                1.0  ...   
11                             1.0                                0.0  ...   
12                             1.0                                     ...   
13                             0.0                                     ...   
14                                  1.0                                ...   
15                                  1.0                                ...   
16                                  1.0                                ...   
17                                  0.0                                ...   
18                                       1.0                           ...   
19                                       0.0                           ...   
20                                            1.0                      ...   
21                                            1.0                      ...   
22                                            0.0                      ...   
23                                            0.0  1.0                 ...   
24                                            0.0  0.0                 ...   
25                                            0.0  0.0                 ...   
26                   1.0                                1.0            ...   
27                   1.0                                1.0            ...   
28                   0.0                                0.0            ...   
29                   0.0                                0.0            ...   
30                        1.0                           1.0            ...   
31                        0.0                                0.0       ...   
32                        0.0                                1.0       ...   
33                        0.0                                1.0       ...   
34                             1.0                           0.0       ...   
35                             0.0                           0.0       ...   
36                             0.0                           0.0       ...   
37                                  1.0                           1.0  ...   
38                                  1.0                           0.0  ...   
39                                  0.0                           0.0  ...   
40                                       1.0                           ...   
41                                       1.0                           ...   
42                                       1.0                           ...   
43                                                                     ...   
44                                                 1.0                 ...   
45                                                 0.0                 ...   

set_number             4                   5                           
visiting_setter_row    3    4    5    6    1    2    3    4    5    6  
rally_number                                                           
1                                      

In [988]:
# count all match serve
serve_all_count = df_serve['serving_team'].value_counts()
serve_all_count

Bradley    101
EIU         98
Name: serving_team, dtype: int64

In [989]:
# serving count by team
serve_all_sum = serve_all_count.sum()
serve_all_sum

199

In [990]:
# count home all match serve
serve_all_count_h = df_serve_h['serving_team'].count()
serve_all_count_h

98

In [991]:
# count visit all match serve
serve_all_count_v = df_serve_v['serving_team'].count()
serve_all_count_v

101

In [992]:
# pivot_all_serve_h > home rotations serves vs opp
pivot_all_serve_h = df_serve.pivot_table("skill", index="home_setter_row", columns="serving_team", aggfunc="count")
pivot_all_serve_h['Diff'] = pivot_all_serve_h[f"{h_name}"]-pivot_all_serve_h[f"{v_name}"]
pivot_all_serve_h

serving_team,Bradley,EIU,Diff
home_setter_row,,,
1,20,21,1
2,24,19,-5
3,18,14,-4
4,16,13,-3
5,11,19,8
6,12,12,0


In [993]:
# pivot_all_serve_v > VISIT difference serving vs receive by rotation
pivot_all_serve_v = df_serve.pivot_table("skill", index="visiting_setter_row", columns="serving_team", aggfunc="count")
pivot_all_serve_v['Diff'] = pivot_all_serve_v[f"{v_name}"]-pivot_all_serve_v[f"{h_name}"]
pivot_all_serve_v

serving_team,Bradley,EIU,Diff
visiting_setter_row,,,
1,16,20,-4
2,22,16,6
3,21,14,7
4,14,16,-2
5,18,15,3
6,10,17,-7


In [994]:
# serving sequence by point
serve_filtered_df = df[df['skill'] == 'Serve'][['set_number','rally_number','serving_team', 'setter_row', 'home_setter_row', 'visiting_setter_row']]
serve_filtered_df

,set_number,rally_number,serving_team,setter_row,home_setter_row,visiting_setter_row
4,1,1,EIU,1,1,1
11,1,2,Bradley,2,1,2
18,1,3,EIU,2,2,2
27,1,4,EIU,2,2,2
34,1,5,Bradley,3,2,3
...,...,...,...,...,...,...
1629,5,23,Bradley,1,6,1
1641,5,24,Bradley,1,6,1
1650,5,25,Bradley,1,6,1
1659,5,26,Bradley,1,6,1


# 9 Receiving rallies

In [995]:
# receive all
df_recep = df[df['skill']== 'Reception']
# df_recep

In [996]:
# HOME all receive
df_recep_h = df_recep[df_recep['receiving_team']== h_name]
# df_recep_h

In [997]:
# VISIT all receive
df_recep_v = df_recep[df_recep['receiving_team']== v_name]
# df_recep_v

In [998]:
# count all match receptions
recep_all_count = df_recep['team'].count()
recep_all_count

184

In [999]:
# HOME count all match receptions
recep_all_count_h = df_recep_h['receiving_team'].count()
recep_all_count_h

92

In [1000]:
# VISIT count all match receptions
recep_all_count_v = df_recep_v['receiving_team'].count()
recep_all_count_v

92

# 10 TOUCHES-Player Individual Stats

In [1001]:
df_h = df[df['team'] == h_name]
# df_h

In [1002]:
df_v = df[df['team'] == v_name]
# df_v

In [1003]:
# HOME group skills win-lost by players
group_df_h = df_h.groupby(['year', 'season','game_number','player_number', 'player_name', 'team', 'skill', 'evaluation_code', 'point_won_by'])['skill'].count().reset_index(name='count')
group_df_h

,year,season,game_number,player_number,player_name,team,skill,evaluation_code,point_won_by,count
0,2024,non-conference,7,2,Laura Paniagua,EIU,Dig,!,Bradley,1
1,2024,non-conference,7,2,Laura Paniagua,EIU,Dig,!,EIU,1
2,2024,non-conference,7,2,Laura Paniagua,EIU,Dig,#,EIU,1
3,2024,non-conference,7,2,Laura Paniagua,EIU,Dig,+,Bradley,3
4,2024,non-conference,7,2,Laura Paniagua,EIU,Dig,+,EIU,5
...,...,...,...,...,...,...,...,...,...,...
239,2024,non-conference,7,23,Sylvia Hasz,EIU,Set,+,EIU,52
240,2024,non-conference,7,23,Sylvia Hasz,EIU,Set,-,Bradley,12
241,2024,non-conference,7,23,Sylvia Hasz,EIU,Set,-,EIU,1
242,2024,non-conference,7,23,Sylvia Hasz,EIU,Set,/,Bradley,2


In [1004]:
# VISIT group skills win-lost by players
group_df_v = df_v.groupby(['year', 'season','game_number','player_number', 'player_name', 'team', 'skill', 'evaluation_code', 'point_won_by'])['skill'].count().reset_index(name='count')
group_df_v

,year,season,game_number,player_number,player_name,team,skill,evaluation_code,point_won_by,count
0,2024,non-conference,7,1,Corrie Brown,Bradley,Block,=,EIU,1
1,2024,non-conference,7,1,Corrie Brown,Bradley,Dig,+,Bradley,1
2,2024,non-conference,7,1,Corrie Brown,Bradley,Dig,+,EIU,1
3,2024,non-conference,7,1,Corrie Brown,Bradley,Dig,=,EIU,2
4,2024,non-conference,7,1,Corrie Brown,Bradley,Serve,!,Bradley,1
...,...,...,...,...,...,...,...,...,...,...
179,2024,non-conference,7,25,Dakota Elliott,Bradley,Block,+,Bradley,3
180,2024,non-conference,7,25,Dakota Elliott,Bradley,Block,=,EIU,3
181,2024,non-conference,7,25,Dakota Elliott,Bradley,Dig,+,Bradley,1
182,2024,non-conference,7,25,Dakota Elliott,Bradley,Dig,+,EIU,1


## Home players stats

In [1005]:
df_h['skill'].unique()

array(['Serve', 'Block', 'Reception', 'Set', 'Attack', 'Dig', 'Freeball'],
      dtype=object)

In [1006]:
# HOME skills by player
skill_h = (
    df_h
    .groupby(['season','year', 'game_number','player_name', 'team'])
    .agg(touches=('skill', 'count'),
        win=('point_won_by', lambda x: x.eq(f"{h_name}").sum(skipna=True)),
        win_pct=('point_won_by', lambda x: round((x.eq(f'{h_name}').sum(skipna=True) / x.count()), 3)))
    .reset_index()
    .sort_values(by='win_pct', ascending=False)
    .reset_index(drop=True)
    .to_string()
)
# Convert to string for printing
print(skill_h)

            season  year  game_number        player_name team  touches  win  win_pct
0   non-conference  2024            7         Karli Mann  EIU       38   26    0.684
1   non-conference  2024            7        Julia Udell  EIU       21   12    0.571
2   non-conference  2024            7       Julia Stanev  EIU       37   20    0.541
3   non-conference  2024            7  Matilda Dahlstrom  EIU      113   59    0.522
4   non-conference  2024            7       Tori Mohesky  EIU       87   45    0.517
5   non-conference  2024            7                  0  EIU        2    1    0.500
6   non-conference  2024            7        Sylvia Hasz  EIU      178   89    0.500
7   non-conference  2024            7     Laura Paniagua  EIU       57   22    0.386
8   non-conference  2024            7    Ariadna Pereles  EIU       54   19    0.352
9   non-conference  2024            7   Destiny Skerrett  EIU       37   10    0.270
10  non-conference  2024            7    Emma Schroederr  EIU    

In [1007]:
# HOME hitting by players
hitting_h = (
    df_h[df_h['skill'] == 'Attack']
    .groupby(['season','year', 'game_number','player_name', 'team', 'skill'])
    .agg(Att=('skill', 'count'),
         K=('evaluation_code', lambda x: x.eq('#').sum(skipna=True)),
         pos=('evaluation_code', lambda x: x.eq('+').sum(skipna=True)),
         ret=('evaluation_code', lambda x: x.eq('!').sum(skipna=True)),
         neg=('evaluation_code', lambda x: x.eq('-').sum(skipna=True)),
         blk=('evaluation_code', lambda x: x.eq('/').sum(skipna=True)),
         err=('evaluation_code', lambda x: x.eq('=').sum(skipna=True)),
        win=('point_won_by', lambda x: x.eq(f"{h_name}").sum(skipna=True)),
        K_pct=('evaluation_code', lambda x: round((x.eq('#').sum(skipna=True) / x.count()), 3)),
        win_pct=('point_won_by', lambda x: round((x.eq(f'{h_name}').sum(skipna=True) / x.count()), 3)))
    .reset_index()
    .sort_values(by='win_pct', ascending=False)
    .reset_index(drop=True)
    .to_string()
)
# Convert to string for printing
print(hitting_h)

           season  year  game_number        player_name team   skill  Att   K  pos  ret  neg  blk  err  win  K_pct  win_pct
0  non-conference  2024            7        Sylvia Hasz  EIU  Attack    9   5    1    2    1    0    0    7  0.556    0.778
1  non-conference  2024            7         Karli Mann  EIU  Attack   30  15    1    2    9    1    2   21  0.500    0.700
2  non-conference  2024            7       Julia Stanev  EIU  Attack   13   6    0    0    5    1    1    9  0.462    0.692
3  non-conference  2024            7  Matilda Dahlstrom  EIU  Attack   43  12    6    4   14    1    6   25  0.279    0.581
4  non-conference  2024            7       Tori Mohesky  EIU  Attack   40  15    2    2   15    2    4   20  0.375    0.500
5  non-conference  2024            7      Lilli Amettis  EIU  Attack    4   1    0    0    2    0    1    1  0.250    0.250
6  non-conference  2024            7   Destiny Skerrett  EIU  Attack    9   0    0    0    7    0    2    1  0.000    0.111
7  non-c

In [1008]:
# HOME Reception by players
rec_h = (
    df_h[df_h['skill'] == 'Reception']
    .groupby(['season','year', 'game_number','player_name', 'team', 'skill'])
    .agg(Att=('skill', 'count'),
         perf=('evaluation_code', lambda x: x.eq('#').sum(skipna=True)),
         pos=('evaluation_code', lambda x: x.eq('+').sum(skipna=True)),
         med=('evaluation_code', lambda x: x.eq('!').sum(skipna=True)),
         neg=('evaluation_code', lambda x: x.eq('-').sum(skipna=True)),
         over=('evaluation_code', lambda x: x.eq('/').sum(skipna=True)),
         err=('evaluation_code', lambda x: x.eq('=').sum(skipna=True)),
        win=('point_won_by', lambda x: x.eq(f"{h_name}").sum(skipna=True)),
        per_pos_pct=('evaluation_code', lambda x: round((x.eq('#').sum(skipna=True) + x.eq('+').sum(skipna=True)) / x.count(), 3) if x.count() > 0 else 0),
        win_pct=('point_won_by', lambda x: round((x.eq(f'{h_name}').sum(skipna=True) / x.count()), 3)))
    .reset_index()
    .sort_values(by='win_pct', ascending=False)
    .reset_index(drop=True)
    .to_string()
)

# Convert to string for printing
print(rec_h)

           season  year  game_number        player_name team      skill  Att  perf  pos  med  neg  over  err  win  per_pos_pct  win_pct
0  non-conference  2024            7        Julia Udell  EIU  Reception    2     0    0    1    1     0    0    2        0.000    1.000
1  non-conference  2024            7       Tori Mohesky  EIU  Reception    5     1    0    2    1     1    0    4        0.200    0.800
2  non-conference  2024            7  Matilda Dahlstrom  EIU  Reception   31     9    3    8    9     1    1   17        0.387    0.548
3  non-conference  2024            7    Ariadna Pereles  EIU  Reception   26     5    5    4    6     3    3   10        0.385    0.385
4  non-conference  2024            7     Laura Paniagua  EIU  Reception   17     4    1    1    6     3    2    6        0.294    0.353
5  non-conference  2024            7   Destiny Skerrett  EIU  Reception    8     2    3    1    2     0    0    2        0.625    0.250
6  non-conference  2024            7      Kaitly

In [1009]:
# HOME Serve by players
serve_h = (
    df_h[df_h['skill'] == 'Serve']
    .groupby(['season','year', 'game_number','player_name', 'team', 'skill'])
    .agg(Att=('skill', 'count'),
         ace=('evaluation_code', lambda x: x.eq('#').sum(skipna=True)),
         pos=('evaluation_code', lambda x: x.eq('+').sum(skipna=True)),
         med=('evaluation_code', lambda x: x.eq('!').sum(skipna=True)),
         neg=('evaluation_code', lambda x: x.eq('-').sum(skipna=True)),
         over=('evaluation_code', lambda x: x.eq('/').sum(skipna=True)),
         err=('evaluation_code', lambda x: x.eq('=').sum(skipna=True)),
        win=('point_won_by', lambda x: x.eq(f"{h_name}").sum(skipna=True)),
        ser_pos_pct=('evaluation_code', lambda x: round((x.eq('#').sum(skipna=True) + x.eq('+').sum(skipna=True) + x.eq('!').sum(skipna=True) + x.eq('/').sum(skipna=True)) / x.count(), 3) if x.count() > 0 else 0),
        win_pct=('point_won_by', lambda x: round((x.eq(f'{h_name}').sum(skipna=True) / x.count()), 3)))
    .reset_index()
    .sort_values(by='win_pct', ascending=False)
    .reset_index(drop=True)
    .to_string()
)

# Convert to string for printing
print(serve_h)

           season  year  game_number        player_name team  skill  Att  ace  pos  med  neg  over  err  win  ser_pos_pct  win_pct
0  non-conference  2024            7       Tori Mohesky  EIU  Serve   21    3    6    1    7     4    0   14        0.667    0.667
1  non-conference  2024            7        Julia Udell  EIU  Serve   13    1    1    2    8     0    1    7        0.308    0.538
2  non-conference  2024            7        Sylvia Hasz  EIU  Serve   21    0    6    5    7     1    2   11        0.571    0.524
3  non-conference  2024            7  Matilda Dahlstrom  EIU  Serve   19    2    4    6    5     0    2    9        0.632    0.474
4  non-conference  2024            7   Destiny Skerrett  EIU  Serve    8    1    2    1    4     0    0    2        0.500    0.250
5  non-conference  2024            7     Laura Paniagua  EIU  Serve    8    0    0    1    6     1    0    2        0.250    0.250
6  non-conference  2024            7    Ariadna Pereles  EIU  Serve    6    0    1 

In [1010]:
# HOME Block by players
block_h = (
    df_h[df_h['skill'] == 'Block']
    .groupby(['season','year', 'game_number','player_name', 'team', 'skill'])
    .agg(Att=('skill', 'count'),
         perf=('evaluation_code', lambda x: x.eq('#').sum(skipna=True)),
         pos=('evaluation_code', lambda x: x.eq('+').sum(skipna=True)),
         med=('evaluation_code', lambda x: x.eq('!').sum(skipna=True)),
         neg=('evaluation_code', lambda x: x.eq('-').sum(skipna=True)),
         over=('evaluation_code', lambda x: x.eq('/').sum(skipna=True)),
         err=('evaluation_code', lambda x: x.eq('=').sum(skipna=True)),
        win=('point_won_by', lambda x: x.eq(f"{h_name}").sum(skipna=True)),
        per_pos_pct=('evaluation_code', lambda x: round((x.eq('#').sum(skipna=True) + x.eq('+').sum(skipna=True) + x.eq('!').sum(skipna=True) + x.eq('-').sum(skipna=True)) / x.count(), 3) if x.count() > 0 else 0),
        win_pct=('point_won_by', lambda x: round((x.eq(f'{h_name}').sum(skipna=True) / x.count()), 3)))
    .reset_index()
    .sort_values(by='win_pct', ascending=False)
    .reset_index(drop=True)
    .to_string()
)

# Convert to string for printing
print(block_h)

           season  year  game_number        player_name team  skill  Att  perf  pos  med  neg  over  err  win  per_pos_pct  win_pct
0  non-conference  2024            7         Karli Mann  EIU  Block    3     1    1    0    0     0    1    2        0.667    0.667
1  non-conference  2024            7  Matilda Dahlstrom  EIU  Block    7     1    3    0    1     0    2    4        0.714    0.571
2  non-conference  2024            7        Sylvia Hasz  EIU  Block   10     3    0    2    0     1    4    5        0.500    0.500
3  non-conference  2024            7       Julia Stanev  EIU  Block   22     4    6    3    1     0    8   10        0.636    0.455
4  non-conference  2024            7    Emma Schroederr  EIU  Block    5     2    0    0    1     0    2    2        0.600    0.400
5  non-conference  2024            7       Ireland Hieb  EIU  Block    4     1    0    0    1     1    1    1        0.500    0.250
6  non-conference  2024            7      Lilli Amettis  EIU  Block    3    

In [1011]:
# HOME Defense by players
dig_h = (
    df_h[df_h['skill'] == 'Dig']
    .groupby(['season','year', 'game_number','player_name', 'team', 'skill'])
    .agg(Att=('skill', 'count'),
         perf=('evaluation_code', lambda x: x.eq('#').sum(skipna=True)),
         pos=('evaluation_code', lambda x: x.eq('+').sum(skipna=True)),
         med=('evaluation_code', lambda x: x.eq('!').sum(skipna=True)),
         neg=('evaluation_code', lambda x: x.eq('-').sum(skipna=True)),
         over=('evaluation_code', lambda x: x.eq('/').sum(skipna=True)),
         err=('evaluation_code', lambda x: x.eq('=').sum(skipna=True)),
        win=('point_won_by', lambda x: x.eq(f"{h_name}").sum(skipna=True)),
        per_pos_pct=('evaluation_code', lambda x: round((x.eq('#').sum(skipna=True) + x.eq('+').sum(skipna=True)) / x.count(), 3) if x.count() > 0 else 0),
        win_pct=('point_won_by', lambda x: round((x.eq(f'{h_name}').sum(skipna=True) / x.count()), 3)))
    .reset_index()
    .sort_values(by='win_pct', ascending=False)
    .reset_index(drop=True)
    .to_string()
)

# Convert to string for printing
print(dig_h)

            season  year  game_number        player_name team skill  Att  perf  pos  med  neg  over  err  win  per_pos_pct  win_pct
0   non-conference  2024            7                  0  EIU   Dig    1     0    0    0    0     1    0    1        0.000    1.000
1   non-conference  2024            7         Karli Mann  EIU   Dig    4     0    3    1    0     0    0    3        0.750    0.750
2   non-conference  2024            7       Julia Stanev  EIU   Dig    2     0    1    0    0     0    1    1        0.500    0.500
3   non-conference  2024            7     Laura Paniagua  EIU   Dig   18     1    8    2    3     0    4    7        0.500    0.389
4   non-conference  2024            7   Destiny Skerrett  EIU   Dig    8     1    3    0    0     0    4    3        0.500    0.375
5   non-conference  2024            7        Sylvia Hasz  EIU   Dig   16     0    7    0    3     0    6    6        0.438    0.375
6   non-conference  2024            7       Tori Mohesky  EIU   Dig    8    

In [1012]:
# HOME setter by players
setter_h = (
    df_h[df_h['skill'] == 'Set']
    .groupby(['season','year', 'game_number','player_name', 'team', 'skill'])
    .agg(Att=('skill', 'count'),
         perf=('evaluation_code', lambda x: x.eq('#').sum(skipna=True)),
         pos=('evaluation_code', lambda x: x.eq('+').sum(skipna=True)),
         med=('evaluation_code', lambda x: x.eq('!').sum(skipna=True)),
         neg=('evaluation_code', lambda x: x.eq('-').sum(skipna=True)),
         over=('evaluation_code', lambda x: x.eq('/').sum(skipna=True)),
         err=('evaluation_code', lambda x: x.eq('=').sum(skipna=True)),
        win=('point_won_by', lambda x: x.eq(f"{h_name}").sum(skipna=True)),
        per_pos_pct=('evaluation_code', lambda x: round((x.eq('#').sum(skipna=True) + x.eq('+').sum(skipna=True)) / x.count(), 3) if x.count() > 0 else 0),
        win_pct=('point_won_by', lambda x: round((x.eq(f'{h_name}').sum(skipna=True) / x.count()), 3)))
    .reset_index()
    .sort_values(by='win_pct', ascending=False)
    .reset_index(drop=True)
    .to_string()
)

# Convert to string for printing
print(setter_h)

           season  year  game_number       player_name team skill  Att  perf  pos  med  neg  over  err  win  per_pos_pct  win_pct
0  non-conference  2024            7      Tori Mohesky  EIU   Set    3     1    2    0    0     0    0    3        1.000    1.000
1  non-conference  2024            7       Sylvia Hasz  EIU   Set  118     2   87   13   13     2    1   60        0.754    0.508
2  non-conference  2024            7   Ariadna Pereles  EIU   Set    6     1    3    1    1     0    0    3        0.667    0.500
3  non-conference  2024            7  Destiny Skerrett  EIU   Set    4     0    1    1    1     0    1    2        0.250    0.500
4  non-conference  2024            7       Julia Udell  EIU   Set    2     0    0    1    0     1    0    1        0.000    0.500
5  non-conference  2024            7    Laura Paniagua  EIU   Set   10     0    9    1    0     0    0    3        0.900    0.300


In [1013]:
# HOME FreeBall by players
free_h = (
    df_h[df_h['skill'] == 'Freeball']
    .groupby(['season','year', 'game_number','player_name', 'team', 'skill'])
    .agg(Att=('skill', 'count'),
         perf=('evaluation_code', lambda x: x.eq('#').sum(skipna=True)),
         pos=('evaluation_code', lambda x: x.eq('+').sum(skipna=True)),
         med=('evaluation_code', lambda x: x.eq('!').sum(skipna=True)),
         neg=('evaluation_code', lambda x: x.eq('-').sum(skipna=True)),
         over=('evaluation_code', lambda x: x.eq('/').sum(skipna=True)),
         err=('evaluation_code', lambda x: x.eq('=').sum(skipna=True)),
        win=('point_won_by', lambda x: x.eq(f"{h_name}").sum(skipna=True)),
        per_pos_pct=('evaluation_code', lambda x: round((x.eq('#').sum(skipna=True) + x.eq('+').sum(skipna=True) + x.eq('!').sum(skipna=True) + x.eq('-').sum(skipna=True)) / x.count(), 3) if x.count() > 0 else 0),
        win_pct=('point_won_by', lambda x: round((x.eq(f'{h_name}').sum(skipna=True) / x.count()), 3)))
    .reset_index()
    .sort_values(by='win_pct', ascending=False)
    .reset_index(drop=True)
    .to_string()
)

# Convert to string for printing
print(free_h)

           season  year  game_number     player_name team     skill  Att  perf  pos  med  neg  over  err  win  per_pos_pct  win_pct
0  non-conference  2024            7     Julia Udell  EIU  Freeball    1     0    1    0    0     0    0    1          1.0      1.0
1  non-conference  2024            7  Laura Paniagua  EIU  Freeball    4     3    1    0    0     0    0    4          1.0      1.0
2  non-conference  2024            7    Tori Mohesky  EIU  Freeball    1     0    1    0    0     0    0    1          1.0      1.0
3  non-conference  2024            7      Karli Mann  EIU  Freeball    1     0    0    1    0     0    0    0          1.0      0.0
4  non-conference  2024            7     Sylvia Hasz  EIU  Freeball    3     0    2    0    1     0    0    0          1.0      0.0


## Visit players stats

In [1014]:
skill_v = (
	df_v
	.groupby(['season','year', 'game_number','player_name', 'team'])
	.agg(touches=('skill', 'count'),
    	win=('point_won_by', lambda x: x.eq(f"{v_name}").sum(skipna=True)),
    	win_pct=('point_won_by', lambda x: round((x.eq(f'{h_name}').sum(skipna=True) / x.count()), 3)))
	.reset_index()
	.sort_values(by='win_pct', ascending=False)
	.reset_index(drop=True)
	.to_string()
)
# Convert to string for printing
print(skill_v)


           season  year  game_number       player_name     team  touches  win  win_pct
0  non-conference  2024            7      Kaylenn Hunt  Bradley       32   12    0.625
1  non-conference  2024            7      Corrie Brown  Bradley       28   11    0.607
2  non-conference  2024            7   Megan Depoorter  Bradley        5    2    0.600
3  non-conference  2024            7       Iva Popovic  Bradley      130   68    0.477
4  non-conference  2024            7    Silan Demikrol  Bradley      108   57    0.472
5  non-conference  2024            7  Irene Mostardini  Bradley       80   43    0.462
6  non-conference  2024            7    Dakota Elliott  Bradley       22   12    0.455
7  non-conference  2024            7       Maria Drapp  Bradley      118   66    0.441
8  non-conference  2024            7       Beste Ayhan  Bradley       31   20    0.355
9  non-conference  2024            7         Anna Kiss  Bradley       35   24    0.314


In [1015]:
# VISIT hitting by players
hitting_v = (
    df_v[df_v['skill'] == 'Attack']
    .groupby(['season','year', 'game_number','player_name', 'team', 'skill'])
    .agg(Att=('skill', 'count'),
         K=('evaluation_code', lambda x: x.eq('#').sum(skipna=True)),
         pos=('evaluation_code', lambda x: x.eq('+').sum(skipna=True)),
         ret=('evaluation_code', lambda x: x.eq('!').sum(skipna=True)),
         neg=('evaluation_code', lambda x: x.eq('-').sum(skipna=True)),
         blk=('evaluation_code', lambda x: x.eq('/').sum(skipna=True)),
         err=('evaluation_code', lambda x: x.eq('=').sum(skipna=True)),
        win=('point_won_by', lambda x: x.eq(f"{v_name}").sum(skipna=True)),
        K_pct=('evaluation_code', lambda x: round((x.eq('#').sum(skipna=True) / x.count()), 3)),
        win_pct=('point_won_by', lambda x: round((x.eq(f'{v_name}').sum(skipna=True) / x.count()), 3)))
    .reset_index()
    .sort_values(by='K_pct', ascending=False)
    .reset_index(drop=True)
    .to_string()
)
print(hitting_v)

           season  year  game_number     player_name     team   skill  Att   K  pos  ret  neg  blk  err  win  K_pct  win_pct
0  non-conference  2024            7       Anna Kiss  Bradley  Attack   28  16    2    1    5    1    3   20  0.571    0.714
1  non-conference  2024            7     Maria Drapp  Bradley  Attack    4   2    0    0    2    0    0    3  0.500    0.750
2  non-conference  2024            7     Beste Ayhan  Bradley  Attack    9   4    0    1    2    1    1    6  0.444    0.667
3  non-conference  2024            7  Dakota Elliott  Bradley  Attack   12   5    0    0    4    0    3    7  0.417    0.583
4  non-conference  2024            7    Kaylenn Hunt  Bradley  Attack   23   9    1    0    9    3    1   10  0.391    0.435
5  non-conference  2024            7     Iva Popovic  Bradley  Attack   48  16    4    3   19    5    1   24  0.333    0.500
6  non-conference  2024            7  Silan Demikrol  Bradley  Attack   39  11    1    1   19    2    5   23  0.282    0.590


In [1016]:
# VISIT Reception by players
rec_v = (
	df_v[df_v['skill'] == 'Reception']
	.groupby(['season','year', 'game_number','player_name', 'team', 'skill'])
	.agg(Att=('skill', 'count'),
     	perf=('evaluation_code', lambda x: x.eq('#').sum(skipna=True)),
     	pos=('evaluation_code', lambda x: x.eq('+').sum(skipna=True)),
     	med=('evaluation_code', lambda x: x.eq('!').sum(skipna=True)),
     	neg=('evaluation_code', lambda x: x.eq('-').sum(skipna=True)),
     	over=('evaluation_code', lambda x: x.eq('/').sum(skipna=True)),
     	err=('evaluation_code', lambda x: x.eq('=').sum(skipna=True)),
    	win=('point_won_by', lambda x: x.eq(f"{v_name}").sum(skipna=True)),
    	per_pos_pct=('evaluation_code', lambda x: round((x.eq('#').sum(skipna=True) + x.eq('+').sum(skipna=True)) / x.count(), 3) if x.count() > 0 else 0),
    	win_pct=('point_won_by', lambda x: round((x.eq(f'{v_name}').sum(skipna=True) / x.count()), 3)))
	.reset_index()
	.sort_values(by='win_pct', ascending=False)
	.reset_index(drop=True)
	.to_string()
)
print(rec_v)


           season  year  game_number       player_name     team      skill  Att  perf  pos  med  neg  over  err  win  per_pos_pct  win_pct
0  non-conference  2024            7  Irene Mostardini  Bradley  Reception   17     6    3    2    4     1    1   10        0.529    0.588
1  non-conference  2024            7       Iva Popovic  Bradley  Reception   43    11    9    8    8     3    4   21        0.465    0.488
2  non-conference  2024            7    Silan Demikrol  Bradley  Reception   31     7    3    9    8     2    2   15        0.323    0.484
3  non-conference  2024            7    Dakota Elliott  Bradley  Reception    1     0    1    0    0     0    0    0        1.000    0.000


In [1017]:
# VISIT Serve by players
hitting_v = (
	df_v[df_v['skill'] == 'Attack']
	.groupby(['season','year', 'game_number','player_name', 'team', 'skill'])
	.agg(Att=('skill', 'count'),
     	K=('evaluation_code', lambda x: x.eq('#').sum(skipna=True)),
     	pos=('evaluation_code', lambda x: x.eq('+').sum(skipna=True)),
     	ret=('evaluation_code', lambda x: x.eq('!').sum(skipna=True)),
     	neg=('evaluation_code', lambda x: x.eq('-').sum(skipna=True)),
     	blk=('evaluation_code', lambda x: x.eq('/').sum(skipna=True)),
     	err=('evaluation_code', lambda x: x.eq('=').sum(skipna=True)),
    	win=('point_won_by', lambda x: x.eq(f"{v_name}").sum(skipna=True)),
    	K_pct=('evaluation_code', lambda x: round((x.eq('#').sum(skipna=True) / x.count()), 3)),
    	win_pct=('point_won_by', lambda x: round((x.eq(f'{v_name}').sum(skipna=True) / x.count()), 3)))
	.reset_index()
	.sort_values(by='K_pct', ascending=False)
	.reset_index(drop=True)
	.to_string()
)
print(hitting_v)


           season  year  game_number     player_name     team   skill  Att   K  pos  ret  neg  blk  err  win  K_pct  win_pct
0  non-conference  2024            7       Anna Kiss  Bradley  Attack   28  16    2    1    5    1    3   20  0.571    0.714
1  non-conference  2024            7     Maria Drapp  Bradley  Attack    4   2    0    0    2    0    0    3  0.500    0.750
2  non-conference  2024            7     Beste Ayhan  Bradley  Attack    9   4    0    1    2    1    1    6  0.444    0.667
3  non-conference  2024            7  Dakota Elliott  Bradley  Attack   12   5    0    0    4    0    3    7  0.417    0.583
4  non-conference  2024            7    Kaylenn Hunt  Bradley  Attack   23   9    1    0    9    3    1   10  0.391    0.435
5  non-conference  2024            7     Iva Popovic  Bradley  Attack   48  16    4    3   19    5    1   24  0.333    0.500
6  non-conference  2024            7  Silan Demikrol  Bradley  Attack   39  11    1    1   19    2    5   23  0.282    0.590


In [1018]:
# VISIT Block by players
block_v = (
	df_v[df_v['skill'] == 'Block']
	.groupby(['season','year', 'game_number','player_name', 'team', 'skill'])
	.agg(Att=('skill', 'count'),
     	perf=('evaluation_code', lambda x: x.eq('#').sum(skipna=True)),
     	pos=('evaluation_code', lambda x: x.eq('+').sum(skipna=True)),
     	med=('evaluation_code', lambda x: x.eq('!').sum(skipna=True)),
     	neg=('evaluation_code', lambda x: x.eq('-').sum(skipna=True)),
     	over=('evaluation_code', lambda x: x.eq('/').sum(skipna=True)),
     	err=('evaluation_code', lambda x: x.eq('=').sum(skipna=True)),
    	win=('point_won_by', lambda x: x.eq(f"{v_name}").sum(skipna=True)),
    	per_pos_pct=('evaluation_code', lambda x: round((x.eq('#').sum(skipna=True) + x.eq('+').sum(skipna=True) + x.eq('!').sum(skipna=True) + x.eq('-').sum(skipna=True)) / x.count(), 3) if x.count() > 0 else 0),
    	win_pct=('point_won_by', lambda x: round((x.eq(f'{v_name}').sum(skipna=True) / x.count()), 3)))
	.reset_index()
	.sort_values(by='win_pct', ascending=False)
	.reset_index(drop=True)
	.to_string()
)

# Convert to string for printing
print(block_v)


           season  year  game_number     player_name     team  skill  Att  perf  pos  med  neg  over  err  win  per_pos_pct  win_pct
0  non-conference  2024            7     Iva Popovic  Bradley  Block    1     0    0    1    0     0    0    1        1.000    1.000
1  non-conference  2024            7     Beste Ayhan  Bradley  Block   15     4    6    4    0     0    1   11        0.933    0.733
2  non-conference  2024            7  Dakota Elliott  Bradley  Block    7     0    3    1    0     0    3    4        0.571    0.571
3  non-conference  2024            7       Anna Kiss  Bradley  Block    6     2    0    2    0     0    2    3        0.667    0.500
4  non-conference  2024            7  Silan Demikrol  Bradley  Block    4     0    1    0    0     0    3    1        0.250    0.250
5  non-conference  2024            7    Kaylenn Hunt  Bradley  Block    8     0    2    2    1     0    3    1        0.625    0.125
6  non-conference  2024            7    Corrie Brown  Bradley  Block 

In [1019]:
# VISIT Defense by players
dig_v = (
	df_v[df_v['skill'] == 'Dig']
	.groupby(['season','year', 'game_number','player_name', 'team', 'skill'])
	.agg(Att=('skill', 'count'),
     	perf=('evaluation_code', lambda x: x.eq('#').sum(skipna=True)),
     	pos=('evaluation_code', lambda x: x.eq('+').sum(skipna=True)),
     	med=('evaluation_code', lambda x: x.eq('!').sum(skipna=True)),
     	neg=('evaluation_code', lambda x: x.eq('-').sum(skipna=True)),
     	over=('evaluation_code', lambda x: x.eq('/').sum(skipna=True)),
     	err=('evaluation_code', lambda x: x.eq('=').sum(skipna=True)),
    	win=('point_won_by', lambda x: x.eq(f"{v_name}").sum(skipna=True)),
    	per_pos_pct=('evaluation_code', lambda x: round((x.eq('#').sum(skipna=True) + x.eq('+').sum(skipna=True)) / x.count(), 3) if x.count() > 0 else 0),
    	win_pct=('point_won_by', lambda x: round((x.eq(f'{v_name}').sum(skipna=True) / x.count()), 3)))
	.reset_index()
	.sort_values(by='win_pct', ascending=False)
	.reset_index(drop=True)
	.to_string()
)

# Convert to string for printing
print(dig_v)


           season  year  game_number       player_name     team skill  Att  perf  pos  med  neg  over  err  win  per_pos_pct  win_pct
0  non-conference  2024            7         Anna Kiss  Bradley   Dig    1     0    1    0    0     0    0    1        1.000    1.000
1  non-conference  2024            7      Kaylenn Hunt  Bradley   Dig    1     0    0    0    0     1    0    1        0.000    1.000
2  non-conference  2024            7   Megan Depoorter  Bradley   Dig    1     0    1    0    0     0    0    1        1.000    1.000
3  non-conference  2024            7       Iva Popovic  Bradley   Dig   16     0   10    1    2     0    3   11        0.625    0.688
4  non-conference  2024            7    Dakota Elliott  Bradley   Dig    2     0    2    0    0     0    0    1        1.000    0.500
5  non-conference  2024            7  Irene Mostardini  Bradley   Dig   28     0   17    0    3     1    7   11        0.607    0.393
6  non-conference  2024            7    Silan Demikrol  Bradle

In [1020]:
# Visit setter by players
setter_v = (
    df_v[df_v['skill'] == 'Set']
    .groupby(['season','year', 'game_number','player_name', 'team', 'skill'])
    .agg(Att=('skill', 'count'),
         perf=('evaluation_code', lambda x: x.eq('#').sum(skipna=True)),
         pos=('evaluation_code', lambda x: x.eq('+').sum(skipna=True)),
         med=('evaluation_code', lambda x: x.eq('!').sum(skipna=True)),
         neg=('evaluation_code', lambda x: x.eq('-').sum(skipna=True)),
         over=('evaluation_code', lambda x: x.eq('/').sum(skipna=True)),
         err=('evaluation_code', lambda x: x.eq('=').sum(skipna=True)),
        win=('point_won_by', lambda x: x.eq(f"{h_name}").sum(skipna=True)),
        per_pos_pct=('evaluation_code', lambda x: round((x.eq('#').sum(skipna=True) + x.eq('+').sum(skipna=True)) / x.count(), 3) if x.count() > 0 else 0),
        win_pct=('point_won_by', lambda x: round((x.eq(f'{h_name}').sum(skipna=True) / x.count()), 3)))
    .reset_index()
    .sort_values(by='win_pct', ascending=False)
    .reset_index(drop=True)
    .to_string()
)

# Convert to string for printing
print(setter_v)

           season  year  game_number       player_name     team skill  Att  perf  pos  med  neg  over  err  win  per_pos_pct  win_pct
0  non-conference  2024            7    Silan Demikrol  Bradley   Set    1     0    0    0    1     0    0    1        0.000    1.000
1  non-conference  2024            7      Corrie Brown  Bradley   Set    9     0    8    0    0     1    0    4        0.889    0.444
2  non-conference  2024            7       Maria Drapp  Bradley   Set   88     0   85    1    2     0    0   35        0.966    0.398
3  non-conference  2024            7  Irene Mostardini  Bradley   Set   10     0   10    0    0     0    0    3        1.000    0.300


In [1021]:
# VISIT FreeBall by players
free_v = (
	df_v[df_v['skill'] == 'Freeball']
	.groupby(['season','year', 'game_number','player_name', 'team', 'skill'])
	.agg(Att=('skill', 'count'),
     	perf=('evaluation_code', lambda x: x.eq('#').sum(skipna=True)),
     	pos=('evaluation_code', lambda x: x.eq('+').sum(skipna=True)),
     	med=('evaluation_code', lambda x: x.eq('!').sum(skipna=True)),
     	neg=('evaluation_code', lambda x: x.eq('-').sum(skipna=True)),
     	over=('evaluation_code', lambda x: x.eq('/').sum(skipna=True)),
     	err=('evaluation_code', lambda x: x.eq('=').sum(skipna=True)),
    	win=('point_won_by', lambda x: x.eq(f"{v_name}").sum(skipna=True)),
    	per_pos_pct=('evaluation_code', lambda x: round((x.eq('#').sum(skipna=True) + x.eq('+').sum(skipna=True) + x.eq('!').sum(skipna=True) + x.eq('-').sum(skipna=True)) / x.count(), 3) if x.count() > 0 else 0),
    	win_pct=('point_won_by', lambda x: round((x.eq(f'{v_name}').sum(skipna=True) / x.count()), 3)))
	.reset_index()
	.sort_values(by='win_pct', ascending=False)
	.reset_index(drop=True)
	.to_string()
)

# Convert to string for printing
print(free_v)


           season  year  game_number       player_name     team     skill  Att  perf  pos  med  neg  over  err  win  per_pos_pct  win_pct
0  non-conference  2024            7    Silan Demikrol  Bradley  Freeball    1     0    1    0    0     0    0    1          1.0      1.0
1  non-conference  2024            7  Irene Mostardini  Bradley  Freeball    5     0    5    0    0     0    0    3          1.0      0.6
2  non-conference  2024            7       Iva Popovic  Bradley  Freeball    1     0    1    0    0     0    0    0          1.0      0.0


# 11 Set 1


In [1022]:
# set1 df all serve
set_1_serve_all = df_serve[df_serve['set_number'] == 1]
set_1_serve_all

,match_id,team,player_number,player_name,player_id,skill,evaluation_code,setter_position,attack_code,set_code,...,possesion_number,point_id,home_setter_row,setter_row,visiting_setter_row,year,game_number,season,date,color
4,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,23,Sylvia Hasz,HAS-SYL,Serve,-,1,0,0,...,0,1,1,1,1,2024,7,non-conference,2024-09-14,cyan
11,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,11,Iva Popovic,-515751,Serve,-,6,0,0,...,0,6,1,2,2,2024,7,non-conference,2024-09-14,blue
18,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,10,Matilda Dahlstrom,DAL-MAT,Serve,!,6,0,0,...,0,11,2,2,2,2024,7,non-conference,2024-09-14,blue
27,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,10,Matilda Dahlstrom,DAL-MAT,Serve,+,6,0,0,...,0,19,2,2,2,2024,7,non-conference,2024-09-14,cyan
34,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,8,Irene Mostardini,-480052,Serve,+,5,0,0,...,0,23,2,3,3,2024,7,non-conference,2024-09-14,cyan
48,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,8,Irene Mostardini,-480052,Serve,-,5,0,0,...,0,36,2,3,3,2024,7,non-conference,2024-09-14,cyan
54,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,8,Irene Mostardini,-480052,Serve,-,5,0,0,...,0,40,2,3,3,2024,7,non-conference,2024-09-14,cyan
60,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,8,Irene Mostardini,-480052,Serve,-,5,0,0,...,0,45,2,3,3,2024,7,non-conference,2024-09-14,cyan
69,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,8,Irene Mostardini,-480052,Serve,-,5,0,0,...,0,52,2,3,3,2024,7,non-conference,2024-09-14,cyan
76,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,8,Irene Mostardini,-480052,Serve,-,5,0,0,...,0,57,2,3,3,2024,7,non-conference,2024-09-14,cyan


In [1023]:
# set_1_serve_teams > set1 total serves by teams
set_1_serve_teams = set_1_serve_all['serving_team'].value_counts()
set_1_serve_teams

Bradley    24
EIU        18
Name: serving_team, dtype: int64

In [1024]:
# SUM all serves set 1
set_1_all_serve = set_1_serve_teams.sum()
set_1_all_serve

42

In [1025]:
# set 1 calculation 
def set1_calc(set_1_all_serve):
  if set_1_all_serve <= 48:
    return (1)
  elif set_1_all_serve > 48:
    return (set_1_all_serve / 48)
  else:
    return (0)

set1_calculation = set1_calc(set_1_all_serve)

set1_calculation

1

In [1026]:
# Home pivot set 1 by rotation
pivot_set1_serve_h = set_1_serve_all.pivot_table("skill", index="home_setter_row", columns="serving_team", aggfunc="count")
pivot_set1_serve_h['Diff'] = pivot_set1_serve_h[f"{h_name}"]-pivot_set1_serve_h[f"{v_name}"]
pivot_set1_serve_h

serving_team,Bradley,EIU,Diff
home_setter_row,,,
1,2.0,3.0,1.0
2,9.0,5.0,-4.0
3,6.0,4.0,-2.0
4,5.0,2.0,-3.0
5,NaN,1.0,NaN
6,2.0,3.0,1.0


In [1027]:
# VISIT pivot set 1 by rotation
pivot_set1_serve_v = set_1_serve_all.pivot_table("skill", index="visiting_setter_row", columns="serving_team", aggfunc="count")
pivot_set1_serve_v['Diff'] = pivot_set1_serve_v[f"{v_name}"]-pivot_set1_serve_v[f"{h_name}"]
pivot_set1_serve_v

serving_team,Bradley,EIU,Diff
visiting_setter_row,,,
1,2,3,-1
2,2,5,-3
3,9,3,6
4,5,2,3
5,4,1,3
6,2,4,-2


# 12 Set 2

In [1028]:
# set2 df all serve
set_2_serve_all = df_serve[df_serve['set_number'] == 2]
set_2_serve_all

,match_id,team,player_number,player_name,player_id,skill,evaluation_code,setter_position,attack_code,set_code,...,possesion_number,point_id,home_setter_row,setter_row,visiting_setter_row,year,game_number,season,date,color
346,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,12,Maria Drapp,-515752,Serve,!,1,0,0,...,0,249,1,1,1,2024,7,non-conference,2024-09-14,blue
355,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,10,Matilda Dahlstrom,DAL-MAT,Serve,!,6,0,0,...,0,255,2,2,1,2024,7,non-conference,2024-09-14,blue
369,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,10,Matilda Dahlstrom,DAL-MAT,Serve,!,6,0,0,...,0,267,2,2,1,2024,7,non-conference,2024-09-14,blue
377,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,10,Matilda Dahlstrom,DAL-MAT,Serve,+,6,0,0,...,0,273,2,2,1,2024,7,non-conference,2024-09-14,cyan
398,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,11,Iva Popovic,-515751,Serve,!,6,0,0,...,0,292,2,2,2,2024,7,non-conference,2024-09-14,blue
408,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,2,Laura Paniagua,LAU-PAN,Serve,-,5,0,0,...,0,300,3,3,2,2024,7,non-conference,2024-09-14,cyan
427,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,8,Irene Mostardini,-480052,Serve,/,5,0,0,...,0,317,3,3,3,2024,7,non-conference,2024-09-14,cyan
434,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,8,Irene Mostardini,-480052,Serve,-,5,0,0,...,0,322,3,3,3,2024,7,non-conference,2024-09-14,blue
441,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,6,Destiny Skerrett,SKE-DES,Serve,!,4,0,0,...,0,327,4,4,3,2024,7,non-conference,2024-09-14,cyan
448,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,1,Corrie Brown,-433745,Serve,+,4,0,0,...,0,332,4,4,4,2024,7,non-conference,2024-09-14,blue


In [1029]:
# set_2_serve_teams > set1 total serves by teams
set_2_serve_teams = set_2_serve_all['serving_team'].value_counts()
set_2_serve_teams

EIU        24
Bradley    21
Name: serving_team, dtype: int64

In [1030]:
# set_2_all_serve > SUM all serves set 2
set_2_all_serve = set_2_serve_teams.sum()
set_2_all_serve

45

In [1031]:
# set 2 calculation 
def set2_calc(set_2_all_serve):
  if set_2_all_serve <= 48:
    return (1)
  elif set_2_all_serve > 48:
    return (set_2_all_serve / 48)
  else:
    return (0)

set2_calculation = set2_calc(set_2_all_serve)

set2_calculation

1

In [1032]:
# Home pivot set 2 by rotation
pivot_set2_serve_h = set_2_serve_all.pivot_table("skill", index="home_setter_row", columns="serving_team", aggfunc="count")
pivot_set2_serve_h['Diff'] = pivot_set2_serve_h[f"{h_name}"]-pivot_set2_serve_h[f"{v_name}"]
pivot_set2_serve_h

serving_team,Bradley,EIU,Diff
home_setter_row,,,
1,3,5,2
2,3,6,3
3,5,4,-1
4,2,4,2
5,6,2,-4
6,2,3,1


In [1033]:
# VISIT pivot set 2 by rotation
pivot_set2_serve_v = set_2_serve_all.pivot_table("skill", index="visiting_setter_row", columns="serving_team", aggfunc="count")
pivot_set2_serve_v['Diff'] = pivot_set2_serve_v[f"{v_name}"]-pivot_set2_serve_v[f"{h_name}"]
pivot_set2_serve_v

serving_team,Bradley,EIU,Diff
visiting_setter_row,,,
1,3,6,-3
2,4,3,1
3,4,5,-1
4,2,3,-1
5,6,2,4
6,2,5,-3


# 13 Set 3

In [1034]:
# set3 df all serve
set_3_serve_all = df_serve[df_serve['set_number'] == 3]
set_3_serve_all

,match_id,team,player_number,player_name,player_id,skill,evaluation_code,setter_position,attack_code,set_code,...,possesion_number,point_id,home_setter_row,setter_row,visiting_setter_row,year,game_number,season,date,color
781,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,23,Sylvia Hasz,HAS-SYL,Serve,+,1,0,0,...,0,586,1,1,1,2024,7,non-conference,2024-09-14,cyan
788,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,11,Iva Popovic,-515751,Serve,/,6,0,0,...,0,591,1,2,2,2024,7,non-conference,2024-09-14,cyan
796,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,11,Iva Popovic,-515751,Serve,#,6,0,0,...,0,597,1,2,2,2024,7,non-conference,2024-09-14,cyan
799,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,11,Iva Popovic,-515751,Serve,=,6,0,0,...,0,599,1,2,2,2024,7,non-conference,2024-09-14,blue
803,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,10,Matilda Dahlstrom,DAL-MAT,Serve,!,6,0,0,...,0,600,2,2,2,2024,7,non-conference,2024-09-14,cyan
810,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,8,Irene Mostardini,-480052,Serve,!,5,0,0,...,0,605,2,3,3,2024,7,non-conference,2024-09-14,cyan
822,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,8,Irene Mostardini,-480052,Serve,+,5,0,0,...,0,616,2,3,3,2024,7,non-conference,2024-09-14,cyan
834,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,8,Irene Mostardini,-480052,Serve,-,5,0,0,...,0,627,2,3,3,2024,7,non-conference,2024-09-14,blue
841,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,2,Laura Paniagua,LAU-PAN,Serve,/,5,0,0,...,0,632,3,3,3,2024,7,non-conference,2024-09-14,cyan
855,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,1,Corrie Brown,-433745,Serve,=,4,0,0,...,0,643,3,4,4,2024,7,non-conference,2024-09-14,blue


In [1035]:
# set_3_serve_teams > set3 total serves by teams
set_3_serve_teams = set_3_serve_all['serving_team'].value_counts()
set_3_serve_teams

Bradley    24
EIU        19
Name: serving_team, dtype: int64

In [1036]:
# set_3_all_serve > SUM all serves set 3
set_3_all_serve = set_3_serve_teams.sum()
set_3_all_serve

43

In [1037]:
# set 3 calculation 
def set3_calc(set_3_all_serve):
  if set_3_all_serve == 0:
    return (0)
  elif 0 < set_3_all_serve <= 48:
    return (1)
  elif set_3_all_serve > 48:
    return (set_3_all_serve / 48)
  else:
    return (0)
set3_calculation = set3_calc(set_3_all_serve)
set3_calculation

1

In [1038]:
# Home pivot set 3 by rotation
pivot_set3_serve_h = set_3_serve_all.pivot_table("skill", index="home_setter_row", columns="serving_team", aggfunc="count")
pivot_set3_serve_h['Diff'] = pivot_set3_serve_h[f"{h_name}"]-pivot_set3_serve_h[f"{v_name}"]
pivot_set3_serve_h

serving_team,Bradley,EIU,Diff
home_setter_row,,,
1,8,3,-5
2,4,3,-1
3,2,2,0
4,5,2,-3
5,3,7,4
6,2,2,0


In [1039]:
# VISIT pivot set 3 by rotation
pivot_set3_serve_v = set_3_serve_all.pivot_table("skill", index="visiting_setter_row", columns="serving_team", aggfunc="count")
pivot_set3_serve_v['Diff'] = pivot_set3_serve_v[f"{v_name}"]-pivot_set3_serve_v[f"{h_name}"]
pivot_set3_serve_v

serving_team,Bradley,EIU,Diff
visiting_setter_row,,,
1,2,3,-1
2,8,3,5
3,4,2,2
4,2,2,0
5,5,7,-2
6,3,2,1


# 14 Set 4

In [1040]:
# set4 df all serve
set_4_serve_all = df_serve[df_serve['set_number'] == 4]
set_4_serve_all

,match_id,team,player_number,player_name,player_id,skill,evaluation_code,setter_position,attack_code,set_code,...,possesion_number,point_id,home_setter_row,setter_row,visiting_setter_row,year,game_number,season,date,color
1133,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,12,Maria Drapp,-515752,Serve,#,1,0,0,...,0,836,1,1,1,2024,7,non-conference,2024-09-14,cyan
1136,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,12,Maria Drapp,-515752,Serve,-,1,0,0,...,0,838,1,1,1,2024,7,non-conference,2024-09-14,blue
1143,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,10,Matilda Dahlstrom,DAL-MAT,Serve,#,6,0,0,...,0,843,2,2,1,2024,7,non-conference,2024-09-14,blue
1146,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,10,Matilda Dahlstrom,DAL-MAT,Serve,-,6,0,0,...,0,845,2,2,1,2024,7,non-conference,2024-09-14,cyan
1156,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,11,Iva Popovic,-515751,Serve,!,6,0,0,...,0,853,2,2,2,2024,7,non-conference,2024-09-14,blue
1163,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,5,Ariadna Pereles,PER-ARI,Serve,!,5,0,0,...,0,857,3,3,2,2024,7,non-conference,2024-09-14,blue
1169,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,5,Ariadna Pereles,PER-ARI,Serve,-,5,0,0,...,0,862,3,3,2,2024,7,non-conference,2024-09-14,cyan
1176,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,8,Irene Mostardini,-480052,Serve,=,5,0,0,...,0,867,3,3,3,2024,7,non-conference,2024-09-14,blue
1181,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,6,Destiny Skerrett,SKE-DES,Serve,+,4,0,0,...,0,868,4,4,3,2024,7,non-conference,2024-09-14,cyan
1191,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,1,Corrie Brown,-433745,Serve,=,4,0,0,...,0,875,4,4,4,2024,7,non-conference,2024-09-14,blue


In [1041]:
# set_4_serve_teams > set4 total serves by teams
set_4_serve_teams = set_4_serve_all['serving_team'].value_counts()
set_4_serve_teams

EIU        24
Bradley    18
Name: serving_team, dtype: int64

In [1042]:
# set_4_all_serve > SUM all serves set 4
set_4_all_serve = set_4_serve_teams.sum()
set_4_all_serve

42

In [1043]:
# set 4 calculation 
def set4_calc(set_4_all_serve):
  if set_4_all_serve == 0:
    return (0)
  elif 0 < set_4_all_serve <= 48:
    return (1)
  elif set_4_all_serve > 48:
    return (set_4_all_serve / 48)
  else:
    return (0)
set4_calculation = set4_calc(set_4_all_serve)
set4_calculation

1

In [1044]:
# Home pivot set 4 by rotation
pivot_set4_serve_h = set_4_serve_all.pivot_table("skill", index="home_setter_row", columns="serving_team", aggfunc="count")
pivot_set4_serve_h['Diff'] = pivot_set4_serve_h[f"{h_name}"]-pivot_set4_serve_h[f"{v_name}"]
pivot_set4_serve_h

serving_team,Bradley,EIU,Diff
home_setter_row,,,
1,5,5,0
2,6,3,-3
3,2,3,1
4,2,3,1
5,1,7,6
6,2,3,1


In [1045]:
# VISIT pivot set 4 by rotation
pivot_set4_serve_v = set_4_serve_all.pivot_table("skill", index="visiting_setter_row", columns="serving_team", aggfunc="count")
pivot_set4_serve_v['Diff'] = pivot_set4_serve_v[f"{v_name}"]-pivot_set4_serve_v[f"{h_name}"]
pivot_set4_serve_v

serving_team,Bradley,EIU,Diff
visiting_setter_row,,,
1,5,3,2
2,6,3,3
3,2,3,-1
4,2,7,-5
5,1,3,-2
6,2,5,-3


# 15 Set 5

In [1046]:
# set5 df all serve
set_5_serve_all = df_serve[df_serve['set_number'] == 5]
set_5_serve_all

,match_id,team,player_number,player_name,player_id,skill,evaluation_code,setter_position,attack_code,set_code,...,possesion_number,point_id,home_setter_row,setter_row,visiting_setter_row,year,game_number,season,date,color
1460,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,23,Sylvia Hasz,HAS-SYL,Serve,!,1,0,0,...,0,1070,1,1,1,2024,7,non-conference,2024-09-14,blue
1473,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,23,Sylvia Hasz,HAS-SYL,Serve,-,1,0,0,...,0,1081,1,1,1,2024,7,non-conference,2024-09-14,blue
1479,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,23,Sylvia Hasz,HAS-SYL,Serve,-,1,0,0,...,0,1085,1,1,1,2024,7,non-conference,2024-09-14,blue
1493,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,23,Sylvia Hasz,HAS-SYL,Serve,!,1,0,0,...,0,1097,1,1,1,2024,7,non-conference,2024-09-14,cyan
1500,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,11,Iva Popovic,-515751,Serve,+,6,0,0,...,0,1101,1,2,2,2024,7,non-conference,2024-09-14,cyan
1509,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,11,Iva Popovic,-515751,Serve,+,6,0,0,...,0,1109,1,2,2,2024,7,non-conference,2024-09-14,blue
1516,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,10,Matilda Dahlstrom,DAL-MAT,Serve,-,6,0,0,...,0,1114,2,2,2,2024,7,non-conference,2024-09-14,blue
1528,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,EIU,10,Matilda Dahlstrom,DAL-MAT,Serve,+,6,0,0,...,0,1125,2,2,2,2024,7,non-conference,2024-09-14,cyan
1535,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,8,Irene Mostardini,-480052,Serve,-,5,0,0,...,0,1130,2,3,3,2024,7,non-conference,2024-09-14,cyan
1544,0e54080a-bc40-4683-a1b0-5fdee3adb5e9,Bradley,8,Irene Mostardini,-480052,Serve,=,5,0,0,...,0,1138,2,3,3,2024,7,non-conference,2024-09-14,blue


In [1047]:
# set_5_serve_teams > set4 total serves by teams
set_5_serve_teams = set_5_serve_all['serving_team'].value_counts()
set_5_serve_teams

Bradley    14
EIU        13
Name: serving_team, dtype: int64

In [1048]:
# set_5_all_serve > SUM all serves set 5
set_5_all_serve = set_5_serve_teams.sum()
set_5_all_serve

27

In [1049]:
# set 5 calculation 
def set5_calc(set_5_all_serve):
  if set_5_all_serve == 0:
    return (0)
  elif 0 < set_5_all_serve <= 28:
    return (0.6)
  elif set_5_all_serve > 28:
    return (set_5_all_serve / 28)
  else:
    return (0)
set5_calculation = set5_calc(set_5_all_serve)
set5_calculation

0.6

In [1050]:
# Home pivot set 5 by rotation
pivot_set5_serve_h = set_5_serve_all.pivot_table("skill", index="home_setter_row", columns="serving_team", aggfunc="count")
pivot_set5_serve_h['Diff'] = pivot_set5_serve_h[f"{h_name}"]-pivot_set5_serve_h[f"{v_name}"]
pivot_set5_serve_h

serving_team,Bradley,EIU,Diff
home_setter_row,,,
1,2,5,3
2,2,2,0
3,3,1,-2
4,2,2,0
5,1,2,1
6,4,1,-3


In [1051]:
# VISIT pivot set 5 by rotation
pivot_set5_serve_v = set_5_serve_all.pivot_table("skill", index="home_setter_row", columns="serving_team", aggfunc="count")
pivot_set5_serve_v['Diff'] = pivot_set5_serve_v[f"{h_name}"]-pivot_set5_serve_v[f"{v_name}"]
pivot_set5_serve_v

serving_team,Bradley,EIU,Diff
home_setter_row,,,
1,2,5,3
2,2,2,0
3,3,1,-2
4,2,2,0
5,1,2,1
6,4,1,-3


# 16 Serving Bar charts

## all

In [1052]:
# Serving histogram serving team all match                 
# .update_xaxes(categoryorder='total descending')
fig_all_serve_teams = px.histogram(df_serve, x='team', text_auto=True, color="team", color_discrete_map=color_map, width=600)
fig_all_serve_teams.layout.title = 'Serve all match by Team'
fig_all_serve_teams.show()

## set 1

In [1053]:
# Set1 histogram serving                 
# .update_xaxes(categoryorder='total descending')
fig_set_1_serve_teams = px.histogram(set_1_serve_all, x='team', text_auto=True, color="team", color_discrete_map=color_map, width=600)
fig_set_1_serve_teams.layout.title = 'Serve Game 1'
fig_set_1_serve_teams.show()

## set 2

In [1054]:
# Set2 histogram serving                 
# .update_xaxes(categoryorder='total descending')
fig_set_2_serve_teams = px.histogram(set_2_serve_all, x='team', text_auto=True, color="team", color_discrete_map=color_map, width=600)
fig_set_2_serve_teams.layout.title = 'Serve Game 2'
fig_set_2_serve_teams.show()

## set 3

In [1055]:
# Set3 histogram serving                 
# .update_xaxes(categoryorder='total descending')
fig_set_3_serve_teams = px.histogram(set_3_serve_all, x='team', text_auto=True, color="team", color_discrete_map=color_map, width=600)
fig_set_3_serve_teams.layout.title = 'Serve Game 3'
fig_set_3_serve_teams.show()

## set 4

In [1056]:
# Set4 histogram serving                 
# .update_xaxes(categoryorder='total descending')
fig_set_3_serve_teams = px.histogram(set_4_serve_all, x='team', text_auto=True, color="team", color_discrete_map=color_map, width=600)
fig_set_3_serve_teams.layout.title = 'Serve Game 4'
fig_set_3_serve_teams.show()

## set 5

In [1057]:
# Set5 histogram serving                 
# .update_xaxes(categoryorder='total descending')
fig_set_5_serve_teams = px.histogram(set_5_serve_all, x='team', text_auto=True, color="team", color_discrete_map=color_map, width=600)
fig_set_5_serve_teams.layout.title = 'Serve Game 5'
fig_set_5_serve_teams.show()

In [1058]:
# Data Marquette serving by rotations
# data_rot_h = {'Position': [1, 6, 5, 4, 3, 2],
#         'Serves': [-5, -1, -6, 3, 0, 12]}
# df_rot_h = pd.DataFrame(data_rot_h)

# Add a column for bar color based on the value of 'Serves'
# df_rot_h['Color'] = df_rot_h['Serves'].apply(lambda x: 'pink' if x > 0 else 'red')

# Plotting the bar chart
# fig_h = px.bar(df_rot_h, x='Position', y='Serves', color='Color', 
#              title='Marquette Win or Lost Serves by Setter Positions',
#              labels={'Position': 'Setter Positions', 'Serves': 'Win or Lost Serves'},
#              color_discrete_map={'pink': 'pink', 'red': 'red'}, text_auto=True)

# Set black edge outline for each bar
# fig_h.update_traces(marker_line_color='black', marker_line_width=1.5)

# Show the plot
# fig_h.show()

# 17 Grouped

## df all

In [1059]:
# df.info()

In [1060]:
# group by df_all
grouped_df_all = df.groupby(['year', 'season', 'date', 'game_number','team', 'player_number', 'player_name', 'point_won_by', 'skill', 'evaluation_code', 'home_setter_row', 'visiting_setter_row', 'set_number', 'setter_row']).size().reset_index(name='count')
grouped_df_all

,year,season,date,game_number,team,player_number,player_name,point_won_by,skill,evaluation_code,home_setter_row,visiting_setter_row,set_number,setter_row,count
0,2024,non-conference,2024-09-14,7,Bradley,1,Corrie Brown,Bradley,Dig,+,4,5,3,5,1
1,2024,non-conference,2024-09-14,7,Bradley,1,Corrie Brown,Bradley,Serve,!,3,4,1,4,1
2,2024,non-conference,2024-09-14,7,Bradley,1,Corrie Brown,Bradley,Serve,#,3,4,5,4,1
3,2024,non-conference,2024-09-14,7,Bradley,1,Corrie Brown,Bradley,Serve,+,3,4,1,4,1
4,2024,non-conference,2024-09-14,7,Bradley,1,Corrie Brown,Bradley,Serve,+,3,4,5,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,2024,non-conference,2024-09-14,7,EIU,23,Sylvia Hasz,EIU,Set,+,6,1,3,6,1
1061,2024,non-conference,2024-09-14,7,EIU,23,Sylvia Hasz,EIU,Set,+,6,1,5,6,3
1062,2024,non-conference,2024-09-14,7,EIU,23,Sylvia Hasz,EIU,Set,+,6,6,1,6,1
1063,2024,non-conference,2024-09-14,7,EIU,23,Sylvia Hasz,EIU,Set,+,6,6,2,6,2


## serve all

In [1061]:
# grouped_serv_all
grouped_serv_all = df_serve.groupby(['year', 'season', 'date', 'game_number','team', 'player_number', 'player_name', 'point_won_by', 'evaluation_code', 'home_setter_row', 'visiting_setter_row', 'set_number', 'setter_row']).size().reset_index(name='count')
grouped_serv_all

,year,season,date,game_number,team,player_number,player_name,point_won_by,evaluation_code,home_setter_row,visiting_setter_row,set_number,setter_row,count
0,2024,non-conference,2024-09-14,7,Bradley,1,Corrie Brown,Bradley,!,3,4,1,4,1
1,2024,non-conference,2024-09-14,7,Bradley,1,Corrie Brown,Bradley,#,3,4,5,4,1
2,2024,non-conference,2024-09-14,7,Bradley,1,Corrie Brown,Bradley,+,3,4,1,4,1
3,2024,non-conference,2024-09-14,7,Bradley,1,Corrie Brown,Bradley,+,3,4,5,4,1
4,2024,non-conference,2024-09-14,7,Bradley,1,Corrie Brown,Bradley,-,3,4,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,2024,non-conference,2024-09-14,7,EIU,23,Sylvia Hasz,EIU,+,1,6,4,1,3
160,2024,non-conference,2024-09-14,7,EIU,23,Sylvia Hasz,EIU,-,1,1,1,1,1
161,2024,non-conference,2024-09-14,7,EIU,23,Sylvia Hasz,EIU,-,1,1,5,1,2
162,2024,non-conference,2024-09-14,7,EIU,23,Sylvia Hasz,EIU,-,1,6,2,1,2


## receive all

In [1062]:
#grouped_rec_all
grouped_rec_all = df_recep.groupby(['year', 'season', 'date', 'game_number','team', 'player_number', 'player_name', 'point_won_by', 'evaluation_code', 'home_setter_row', 'visiting_setter_row', 'set_number', 'setter_row']).size().reset_index(name='count')
grouped_rec_all

,year,season,date,game_number,team,player_number,player_name,point_won_by,evaluation_code,home_setter_row,visiting_setter_row,set_number,setter_row,count
0,2024,non-conference,2024-09-14,7,Bradley,8,Irene Mostardini,Bradley,!,2,2,1,2,1
1,2024,non-conference,2024-09-14,7,Bradley,8,Irene Mostardini,Bradley,#,4,4,3,4,1
2,2024,non-conference,2024-09-14,7,Bradley,8,Irene Mostardini,Bradley,#,6,5,4,5,1
3,2024,non-conference,2024-09-14,7,Bradley,8,Irene Mostardini,Bradley,#,6,6,3,6,1
4,2024,non-conference,2024-09-14,7,Bradley,8,Irene Mostardini,Bradley,+,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,2024,non-conference,2024-09-14,7,EIU,11,Kaitlyn Flynn,Bradley,#,2,3,1,2,1
169,2024,non-conference,2024-09-14,7,EIU,11,Kaitlyn Flynn,Bradley,+,2,3,1,2,1
170,2024,non-conference,2024-09-14,7,EIU,20,Julia Udell,EIU,!,6,1,3,6,1
171,2024,non-conference,2024-09-14,7,EIU,20,Julia Udell,EIU,-,1,2,3,1,1


## receive home

In [1063]:
# grouped_rec_h
grouped_rec_h = df_recep_h.groupby(['year', 'season', 'date', 'game_number','team', 'player_number', 'player_name', 'point_won_by', 'evaluation_code', 'home_setter_row', 'visiting_setter_row', 'set_number', 'setter_row']).size().reset_index(name='count')
grouped_rec_h

,year,season,date,game_number,team,player_number,player_name,point_won_by,evaluation_code,home_setter_row,visiting_setter_row,set_number,setter_row,count
0,2024,non-conference,2024-09-14,7,EIU,2,Laura Paniagua,Bradley,#,2,3,1,2,2
1,2024,non-conference,2024-09-14,7,EIU,2,Laura Paniagua,Bradley,#,5,5,2,5,1
2,2024,non-conference,2024-09-14,7,EIU,2,Laura Paniagua,Bradley,-,1,2,3,1,1
3,2024,non-conference,2024-09-14,7,EIU,2,Laura Paniagua,Bradley,-,3,4,1,3,1
4,2024,non-conference,2024-09-14,7,EIU,2,Laura Paniagua,Bradley,-,4,5,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,2024,non-conference,2024-09-14,7,EIU,11,Kaitlyn Flynn,Bradley,#,2,3,1,2,1
82,2024,non-conference,2024-09-14,7,EIU,11,Kaitlyn Flynn,Bradley,+,2,3,1,2,1
83,2024,non-conference,2024-09-14,7,EIU,20,Julia Udell,EIU,!,6,1,3,6,1
84,2024,non-conference,2024-09-14,7,EIU,20,Julia Udell,EIU,-,1,2,3,1,1


## receive visit

In [1064]:
# grouped_rec_v
grouped_rec_v = df_recep_v.groupby(['year', 'season', 'date', 'game_number','team', 'player_number', 'player_name', 'point_won_by', 'evaluation_code', 'home_setter_row', 'visiting_setter_row', 'set_number', 'setter_row']).size().reset_index(name='count')
grouped_rec_v

,year,season,date,game_number,team,player_number,player_name,point_won_by,evaluation_code,home_setter_row,visiting_setter_row,set_number,setter_row,count
0,2024,non-conference,2024-09-14,7,Bradley,8,Irene Mostardini,Bradley,!,2,2,1,2,1
1,2024,non-conference,2024-09-14,7,Bradley,8,Irene Mostardini,Bradley,#,4,4,3,4,1
2,2024,non-conference,2024-09-14,7,Bradley,8,Irene Mostardini,Bradley,#,6,5,4,5,1
3,2024,non-conference,2024-09-14,7,Bradley,8,Irene Mostardini,Bradley,#,6,6,3,6,1
4,2024,non-conference,2024-09-14,7,Bradley,8,Irene Mostardini,Bradley,+,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,2024,non-conference,2024-09-14,7,Bradley,23,Silan Demikrol,EIU,-,1,6,4,6,2
83,2024,non-conference,2024-09-14,7,Bradley,23,Silan Demikrol,EIU,/,1,6,4,6,1
84,2024,non-conference,2024-09-14,7,Bradley,23,Silan Demikrol,EIU,=,2,2,1,2,1
85,2024,non-conference,2024-09-14,7,Bradley,23,Silan Demikrol,EIU,=,5,4,4,4,1


## home grouped rows

In [1065]:
# df_serve.info()

In [1066]:
# grouped_rows_h
grouped_rows_h = df_serve.groupby(['year', 'season', 'date', 'game_number','team', 'home_team_id', 'home_team', 'home_setter_row', 'set_number', 'home_p1','home_p6', 'home_p5', 'home_p4', 'home_p3', 'home_p2', 'point_won_by'])['point_won_by'].size().reset_index(name='count')
grouped_rows_h.head(500)

,year,season,date,game_number,team,home_team_id,home_team,home_setter_row,set_number,home_p1,home_p6,home_p5,home_p4,home_p3,home_p2,point_won_by,count
0,2024,non-conference,2024-09-14,7,Bradley,232,EIU,1,1,23,10,17,12,3,20,EIU,1
1,2024,non-conference,2024-09-14,7,Bradley,232,EIU,1,1,23,10,17,14,11,21,EIU,1
2,2024,non-conference,2024-09-14,7,Bradley,232,EIU,1,2,23,10,17,12,3,21,EIU,2
3,2024,non-conference,2024-09-14,7,Bradley,232,EIU,1,2,23,10,17,12,3,22,EIU,1
4,2024,non-conference,2024-09-14,7,Bradley,232,EIU,1,3,23,10,17,6,3,22,Bradley,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2024,non-conference,2024-09-14,7,EIU,232,EIU,6,3,20,23,10,17,6,3,Bradley,1
131,2024,non-conference,2024-09-14,7,EIU,232,EIU,6,3,22,23,10,17,6,3,Bradley,1
132,2024,non-conference,2024-09-14,7,EIU,232,EIU,6,4,20,23,10,17,6,3,Bradley,1
133,2024,non-conference,2024-09-14,7,EIU,232,EIU,6,4,20,23,10,17,6,3,EIU,2


## visit grouped rows

In [1067]:
# grouped_rows_v
grouped_rows_v = df_serve.groupby(['year', 'season', 'date', 'game_number','team', 'visiting_team_id', 'visiting_team', 'visiting_setter_row', 'set_number', 'visiting_p1','visiting_p6', 'visiting_p5', 'visiting_p4', 'visiting_p3', 'visiting_p2', 'point_won_by'])['point_won_by'].size().reset_index(name='count')
grouped_rows_v.head(50)

,year,season,date,game_number,team,visiting_team_id,visiting_team,visiting_setter_row,set_number,visiting_p1,visiting_p6,visiting_p5,visiting_p4,visiting_p3,visiting_p2,point_won_by,count
0,2024,non-conference,2024-09-14,7,Bradley,202,Bradley,1,1,12,11,19,14,23,25,Bradley,1
1,2024,non-conference,2024-09-14,7,Bradley,202,Bradley,1,1,12,11,19,14,23,25,EIU,1
2,2024,non-conference,2024-09-14,7,Bradley,202,Bradley,1,2,12,11,19,14,23,25,EIU,3
3,2024,non-conference,2024-09-14,7,Bradley,202,Bradley,1,3,12,11,25,14,23,19,EIU,2
4,2024,non-conference,2024-09-14,7,Bradley,202,Bradley,1,4,12,11,25,14,23,19,Bradley,3
5,2024,non-conference,2024-09-14,7,Bradley,202,Bradley,1,4,12,11,25,14,23,19,EIU,2
6,2024,non-conference,2024-09-14,7,Bradley,202,Bradley,1,5,12,11,25,14,23,19,Bradley,3
7,2024,non-conference,2024-09-14,7,Bradley,202,Bradley,1,5,12,11,25,14,23,19,EIU,1
8,2024,non-conference,2024-09-14,7,Bradley,202,Bradley,2,1,11,19,14,23,25,12,EIU,2
9,2024,non-conference,2024-09-14,7,Bradley,202,Bradley,2,2,11,19,14,23,25,12,Bradley,2


# 18 Sunburst

## all skills, player, skill, win-lost, evaluation_code

In [1068]:
# all skills, player, skill, win-lost, evaluation_code
fig_df_all = px.sunburst(grouped_df_all,
                  path=['team', 'player_name', 'skill', 'point_won_by', 'evaluation_code'],
                  values='count',
                  title="All Skills by Player | Win vs Lost | Receiving quality",
                  width=700, 
                  height=700,
                  color="point_won_by",
                 color_discrete_map=color_map
                    )
fig_df_all.show()

## serve all, player, win-lost, evaluation

In [1069]:
# serve all, player, win-lost, evaluation
fig_df_serv = px.sunburst(grouped_serv_all,
                  path=['team', 'player_name', 'point_won_by', 'evaluation_code'],
                  values='count',
                  title="Serves by Player | Win vs Lost | Serving quality",
                  width=700, 
                  height=700,
                  color="point_won_by",
                 color_discrete_map=color_map
                    )
fig_df_serv.show()

## serve all, set-number, player, win-lost, evaluation

In [1070]:
# serve all, set-number, player, win-lost, evaluation
fig_serve_set = px.sunburst(grouped_serv_all,
                  path=['team', 'set_number', 'player_name', 'point_won_by', 'evaluation_code'],
                  values='count',
                  title='Serving Team by Sets| Player serving| Win vs Lost by Team| Serve Quality',
                  width=700, 
                  height=700,
                  color='point_won_by',
                color_discrete_map=color_map
                       )
fig_serve_set.show()

## serve all, row, player, win-lost

In [1071]:
# serve all, row, player, win-lost
fig_serve_row = px.sunburst(grouped_serv_all,
                  path=['team', 'setter_row', 'player_name', 'point_won_by', 'evaluation_code'],
                  values='count',
                  title='Serving Team by Rows| Player serving| Win vs Lost by Team| Serve Quality',
                  width=700, 
                  height=700,
                  color='point_won_by',
                color_discrete_map=color_map
                       )
fig_serve_row.show()

## receive all, player, win-lost, evaluation_code

In [1072]:
# receive all, player, win-lost, evaluation_code
fig_rec_all = px.sunburst(grouped_rec_all,
                path=['team', 'player_name', 'point_won_by', 'evaluation_code'],
                  values='count',
                  title="Receiving by Team| Player receiving| Win vs Lost| Receiving quality",
                  width=700, 
                  height=700,
                  color="point_won_by",
                 color_discrete_map=color_map
                    )      

fig_rec_all.show()

## receive by sets, player, win-lost, evaluation_code

In [1073]:
# receive by sets, player, win-lost, evaluation_code
fig_recep_set = px.sunburst(grouped_rec_all,
                  path=['team', 'set_number', 'player_name', 'point_won_by', 'evaluation_code'],
                  values='count',
                  title='Receiving Team by Sets| Player serving| Win vs Lost by Team| Serve Quality',
                  width=700, 
                  height=700,
                  color='point_won_by',
                color_discrete_map=color_map
                       )
fig_recep_set.show()

## receive by rows, player, win-lost, evaluation_code

In [1074]:
# receive by rows, player, win-lost, evaluation_code
fig_rec_row = px.sunburst(grouped_rec_all,
                  path=['team', 'setter_row', 'player_name', 'point_won_by', 'evaluation_code'],
                  values='count',
                  title='Receiving Team by Rows| Player serving| Win vs Lost by Team| Serve Quality',
                  width=700, 
                  height=700,
                  color='point_won_by',
                color_discrete_map=color_map
                       )
fig_rec_row.show()

## home win-lost by rows

In [1075]:
fig_rot_h_row = px.sunburst(grouped_rows_h,
                  path=['home_team', 'home_setter_row', 'point_won_by', 'home_p1', 'home_p6', 'home_p5', 'home_p4', 'home_p3', 'home_p2'],
                  values='count',
                  title= f"{h_name} Setter In & players positions",
                  width=700, 
                  height=700,
                  color='point_won_by',
                color_discrete_map=color_map
                       )
fig_rot_h_row.show()

## visit win-lost by rows

In [1076]:
fig_rot_v_row = px.sunburst(grouped_rows_v,
                  path=['visiting_team', 'visiting_setter_row', 'point_won_by', 'visiting_p1', 'visiting_p6', 'visiting_p5', 'visiting_p4', 'visiting_p3', 'visiting_p2'],
                  values='count',
                  title= f"{v_name} Setter In & players positions",
                  width=700, 
                  height=700,
                  color='point_won_by',
                color_discrete_map=color_map
                       )
fig_rot_v_row.show()

# 19 Cross table: win-lost by rows and players numbers 

In [1077]:
# Cross rows home rotations

cross_row_h = pd.crosstab(
    index=[df_serve['date'], df_serve['season'], df_serve['game_number'], df_serve['home_team'], df_serve['home_setter_row'], df_serve['home_p1'], df_serve['home_p6'], df_serve['home_p5'], df_serve['home_p4'], df_serve['home_p3'], df_serve['home_p2']]
    , columns=df_serve['point_won_by']
    , margins=True
)

cross_row_h

point_won_by                                                                                                     Bradley  \
date       season         game_number home_team home_setter_row home_p1 home_p6 home_p5 home_p4 home_p3 home_p2            
2024-09-14 non-conference 7           EIU       1               23      10      17      6       3       22             3   
                                                                                        12      3       20             3   
                                                                                                        21             1   
                                                                                                        22            11   
                                                                                        14      11      21             1   
                                                2               6       17      12      3       20      23             0   
                                                                                14      3       21      23             0   
                                                                                        11      21      23             2   
                                                                10      17      6       3       22      23             1   
                                                                                12      3       20      23             2   
                                                                                                21      23             3   
                                                                                                22      23            11   
                                                                                14      11      21      23             6   
                                                3               17      5       3       21      23      6              1   
                                                                        12      3       20      23      6              2   
                                                                                        21      23      10             2   
                                                                                        22      23      10            11   
                                                                        14      3       21      23      6              3   
                                                4               5       3       20      23      6       17             3   
                                                                                21      23      10      17             2   
                                                                6       3       22      23      10      17             9   
                                                                12      3       20      23      6       17             1   
                                                                                21      23      10      17             1   
                                                                                22      23      10      17             1   
                                                5               3       20      23      10      17      5              0   
                                                                        21      23      10      17      12             6   
                                                                        22      23      10      17      6              5   
                                                                                                        12             0   
                                                6               20      23      10      17      5       3              2   
                                                                                                6       3              3   
                                

In [1078]:
# visit rotations

cross_row_v = pd.crosstab(
    index=[df_serve['date'], df_serve['season'], df_serve['game_number'],  df_serve['visiting_team'],  df_serve['visiting_setter_row'], df_serve['set_number'], df_serve['visiting_p1'], df_serve['visiting_p6'], df_serve['visiting_p5'], df_serve['visiting_p4'], df_serve['visiting_p3'], df_serve['visiting_p2']]
    , columns=df_serve['point_won_by']
    , margins=True,
)

cross_row_v

point_won_by                                                                                                                                                Bradley  \
date       season         game_number visiting_team visiting_setter_row set_number visiting_p1 visiting_p6 visiting_p5 visiting_p4 visiting_p3 visiting_p2            
2024-09-14 non-conference 7           Bradley       1                   1          12          11          19          14          23          25                 3   
                                                                        2          12          11          19          14          23          25                 2   
                                                                        3          12          11          25          14          23          19                 3   
                                                                        4          12          11          25          14          23          19                 5   
                                                                        5          12          11          25          14          23          19                 5   
                                                    2                   1          11          19          14          23          25          12                 2   
                                                                        2          11          19          14          23          25          12                 4   
                                                                        3          11          25          14          23          19          12                 8   
                                                                        4          11          25          14          23          19          12                 6   
                                                                        5          11          25          14          23          19          12                 2   
                                                    3                   1          19          14          23          25          12          11                 9   
                                                                        2          19          14          23          25          12          11                 5   
                                                                        3          25          14          23          19          12          11                 4   
                                                                        4          25          14          23          19          12          11                 2   
                                                                        5          25          14          23          19          12          11                 2   
                                                    4                   1          14          23          25          12          11          19                 5   
                                                                        2          14          23          25          12          11          19                 1   
                                                                        3          14          23          19          12          11          25                 2   
                                                                        4          14          23          19          12          11          25                 1   
                                                                        5          14          23          19          12          11          25                 3   
                                                    5                   1          23          25          12          11          19          14                 4   
                                                                        2          23          25          12          11          19          14              

# 20 charts point sequence

## home sideout seq by rows

In [1079]:
# Home Setter rotations vs Visit Point sequence

# '#FFFF00', #set 1 - yellow
# '#FF0000',  # Set 1 - Red
# '#4682B4',  # Set 2 - Blue
# '#32CD32',   # Set 3 - Green
# '#FFA500',  # Set 4 orange
# '#FFC0CB'   # Set 5 pink

color_mapp = {
    1 : '#FF0000',  # Set 1 - Red
    2 : '#4682B4',  # Set 2 - Blue
    3 : '#32CD32',   # Set 3 - Green
    4 : '#FFA500',  # Set 4 orange
    5 : '#FFFF00', #set 5 - yellow
}
    

fig_row_serve_h = px.line(df, x="visiting_team_score", y="home_setter_row", color="set_number",
                 color_discrete_map=color_mapp,
                     labels={"set_number": "Set Number"}) 
fig_row_serve_h.update_traces(textposition="bottom right", mode="markers+lines", marker=dict(size=15, line=dict(width=1)))


fig_row_serve_h.update_layout(
        autosize=False,
        width=800,
        height=300,
        title=dict(
            text=(f"{h_name} sideout vs {v_name}`s serve by Row"),
            font=dict(size=20, color='#000000'),
            x=0.45,
            y=0.8
        ),
        xaxis_title=dict(text=v_name, font=dict(size=16)),
        yaxis_title=dict(text=h_name, font=dict(size=16)),
        plot_bgcolor='rgb(200, 255, 255)',
        xaxis=dict(tickfont=dict(size=14, color='#000000')),
        yaxis=dict(tickfont=dict(size=14, color='#000000')),
        legend=dict(x=1.05, y=.5, orientation='v', font=dict(color='#000000')),
        margin=dict(l=40, r=10, t=100, b=50)
        )

fig_row_serve_h.show()

## home point scoring seq by rows

In [1080]:
# df.info()

In [1081]:
# Home Setter rotations vs Visit Point sequence

# '#FFFF00', #set 1 - yellow
# '#FF0000',  # Set 1 - Red
# '#4682B4',  # Set 2 - Blue
# '#32CD32',   # Set 3 - Green
# '#FFA500',  # Set 4 orange
# '#FFC0CB'   # Set 5 pink

color_mapp = {
    1 : '#FF0000',  # Set 1 - Red
    2 : '#4682B4',  # Set 2 - Blue
    3 : '#32CD32',   # Set 3 - Green
    4 : '#FFA500',  # Set 4 orange
    5 : '#FFFF00', #set 5 - yellow
}
    

fig_row_rec_h = px.line(df, x="home_team_score", y="home_setter_row", color="set_number",
                 color_discrete_map=color_mapp,
                     labels={"set_number": "Set Number"}) 
fig_row_rec_h.update_traces(textposition="bottom right", mode="markers+lines", marker=dict(size=15, line=dict(width=1)))


fig_row_rec_h.update_layout(
        autosize=False,
        width=800,
        height=300,
        title=dict(
           text=(f"{h_name} serve vs {v_name}`s sideout by Row"),
            font=dict(size=20, color='#000000'),
            x=0.45,
            y=0.8
        ),
        xaxis_title=dict(text=h_name, font=dict(size=16)),
        yaxis_title=dict(text=h_name, font=dict(size=16)),
        plot_bgcolor='rgb(200, 255, 255)',
        xaxis=dict(tickfont=dict(size=14, color='#000000')),
        yaxis=dict(tickfont=dict(size=14, color='#000000')),
        legend=dict(x=1.05, y=.5, orientation='v', font=dict(color='#000000')),
        margin=dict(l=40, r=10, t=100, b=50)
        )

fig_row_rec_h.show()

In [1082]:
# sending the graph to plotly studio
# py.plot(fig_line_row_h, filename = 'Marquette_24nov2023_pointscoring_by_rotations_win_lost', auto_open=False)

## visit sideout seq by rows

In [1083]:
# visiting Setter rotations vs Visit Point sequence
# Home Setter rotations vs Visit Point sequence

# '#FFFF00', #set 1 - yellow
# '#FF0000',  # Set 1 - Red
# '#4682B4',  # Set 2 - Blue
# '#32CD32',   # Set 3 - Green
# '#FFA500',  # Set 4 orange
# '#FFC0CB'   # Set 5 pink

color_mapp = {
    1 : '#FF0000',  # Set 1 - Red
    2 : '#4682B4',  # Set 2 - Blue
    3 : '#32CD32',   # Set 3 - Green
    4 : '#FFA500',  # Set 4 orange
    5 : '#FFFF00', #set 5 - yellow
}


fig_row_rec_v = px.line(df, x="home_team_score", y="visiting_setter_row", color='set_number',
                 color_discrete_map=color_mapp,
                     labels={"set_number": "Set Number"}) 
fig_row_rec_v.update_traces(textposition="bottom right", mode="markers+lines", marker=dict(size=15, line=dict(width=1)))


fig_row_rec_v.update_layout(
        autosize=False,
        width=800,
        height=300,
        title=dict(
           text=(f"{v_name} sideout vs {h_name}`s serve by Row"),
            font=dict(size=20, color='#000000'),
            x=0.45,
            y=0.8
        ),
        xaxis_title=dict(text=h_name, font=dict(size=16)),
        yaxis_title=dict(text=v_name, font=dict(size=16)),
        plot_bgcolor='rgb(200, 255, 255)',
        xaxis=dict(tickfont=dict(size=14, color='#000000')),
        yaxis=dict(tickfont=dict(size=14, color='#000000')),
        legend=dict(x=1.05, y=.5, orientation='v', font=dict(color='#000000')),
        margin=dict(l=40, r=10, t=100, b=50)
        )

fig_row_rec_v.show()

## visit point scoring seq by rows

In [1084]:
# visiting Setter rotations vs Visit Point sequence
# Home Setter rotations vs Visit Point sequence

# '#FFFF00', #set 1 - yellow
# '#FF0000',  # Set 1 - Red
# '#4682B4',  # Set 2 - Blue
# '#32CD32',   # Set 3 - Green
# '#FFA500',  # Set 4 orange
# '#FFC0CB'   # Set 5 pink

color_mapp = {
    1 : '#FF0000',  # Set 1 - Red
    2 : '#4682B4',  # Set 2 - Blue
    3 : '#32CD32',   # Set 3 - Green
    4 : '#FFA500',  # Set 4 orange
    5 : '#FFFF00', #set 5 - yellow
}


fig_row_serve_v = px.line(df, x="visiting_team_score", y="visiting_setter_row", color='set_number',
                 color_discrete_map=color_mapp,
                     labels={"set_number": "Set Number"}) 
fig_row_serve_v.update_traces(textposition="bottom right", mode="markers+lines", marker=dict(size=15, line=dict(width=1)))


fig_row_serve_v.update_layout(
        autosize=False,
        width=800,
        height=300,
        title=dict(
            text=(f"{v_name} serve vs {h_name}`s sideout by Row"),
            font=dict(size=20, color='#000000'),
            x=0.45,
            y=0.8
        ),
        xaxis_title=dict(text=v_name, font=dict(size=16)),
        yaxis_title=dict(text=v_name, font=dict(size=16)),
        plot_bgcolor='rgb(200, 255, 255)',
        xaxis=dict(tickfont=dict(size=14, color='#000000')),
        yaxis=dict(tickfont=dict(size=14, color='#000000')),
        legend=dict(x=1.05, y=.5, orientation='v', font=dict(color='#000000')),
        margin=dict(l=40, r=10, t=100, b=50)
        )

fig_row_serve_v.show()

In [1085]:
# sending the graph to plotly studio
# py.plot(fig_line_row_h, filename = 'Marquette_24nov2023_pointscoring_by_rotations_win_lost', auto_open=False)

## Point by Point chart 

In [1086]:
# Points Sequence by Set
# '#FFFF00', #set 1 - yellow
# '#FF0000',  # Set 1 - Red
# '#4682B4',  # Set 2 - Blue
# '#32CD32',   # Set 3 - Green
# '#FFA500',  # Set 4 orange
# '#FFC0CB'   # Set 5 pink

color_mapp = {
    1 : '#FF0000',  # Set 1 - Red
    2 : '#4682B4',  # Set 2 - Blue
    3 : '#32CD32',   # Set 3 - Green
    4 : '#FFA500',  # Set 4 orange
    5 : '#FFFF00', #set 5 - yellow
}
    
fig_line_p = px.line(df, x="home_team_score", y="visiting_team_score", color="set_number",  
                         color_discrete_map=color_mapp,
                     labels={"set_number": "Set Number"}) 
fig_line_p.update_traces(textposition="bottom right", mode="markers+lines", marker=dict(size=10, line=dict(width=1)))


fig_line_p.update_layout(
        autosize=False,
        width=600,
        height=600,
        title=dict(
            text=f"{h_name} vs {v_name} Points Sequence by Set",
            font=dict(size=20, color='#000000'),
            x=0.45,
            y=0.9
        ),
        xaxis_title=dict(text=h_name, font=dict(size=16)),
        yaxis_title=dict(text=v_name, font=dict(size=16)),
        plot_bgcolor='rgb(200, 255, 255)',
        xaxis=dict(tickfont=dict(size=14, color='#000000')),
        yaxis=dict(tickfont=dict(size=14, color='#000000')),
        legend=dict(x=1.05, y=.5, orientation='v', font=dict(color='#000000')),
        margin=dict(l=50, r=10, t=100, b=50)
    ),

fig_line_p.update_layout(
        shapes=[
            dict(
                type= 'line',
                yref= 'y', y0=-0, y1= 25,
                xref= 'x', x0=-0, x1= 25
            ),
            
        ])

fig_line_p.show()

# 21 Sanky Chart

## Home


In [1087]:
# Sanky graphic step 1
dft = df_h
segments = dft['player_name'].unique().tolist()
categories = dft['skill'].unique().tolist()
subcategories = dft['point_won_by'].unique().tolist()
labels = segments + subcategories + categories
labels

['Sylvia Hasz',
 'Julia Stanev',
 'Laura Paniagua',
 'Matilda Dahlstrom',
 'Lilli Amettis',
 'Destiny Skerrett',
 'Kaitlyn Flynn',
 'Tori Mohesky',
 'Ireland Hieb',
 'Ariadna Pereles',
 'Julia Udell',
 0,
 'Karli Mann',
 'Emma Schroederr',
 'Bradley',
 'EIU',
 'Serve',
 'Block',
 'Reception',
 'Set',
 'Attack',
 'Dig',
 'Freeball']

In [1088]:
# Sanky graph step 2
sources = []
targets = []
values = []

In [1089]:
# Sanky graph step 3 (player-skill-win)
for segment in segments:
    for subcat in subcategories:
        dft1 = dft.loc[dft['player_name']==segment]
        dft1 = dft1.loc[dft['point_won_by']==subcat,'point_id'].count()
        sources.append(labels.index(segment))
        targets.append(labels.index(subcat))
        values.append(dft1)
        
for categorie in categories:
    for subcat in dft['point_won_by'].unique():
        dft2 = dft.loc[dft['skill']==categorie]
        dft2 = dft2.loc[dft['point_won_by']==subcat,'point_id'].count()
        sources.append(labels.index(subcat))
        targets.append(labels.index(categorie))
        values.append(dft2)

In [1090]:
# Sanky graph step 4
import plotly.graph_objects as go

fig_sanky_h = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = labels,
      color = "blue"
      

    ),
    link = dict(
      source = sources,
      target = targets,
      value = values
  ))])

fig_sanky_h.update_layout(title_text= f"{h_name} relation between Player Skills & Win Rallies", font_size=10)

fig_sanky_h

## Visit

In [1091]:
# Sanky graphic step 1
dft = df_v
segments = dft['player_name'].unique().tolist()
categories = dft['skill'].unique().tolist()
subcategories = dft['point_won_by'].unique().tolist()
labels = segments + subcategories + categories
labels

['Irene Mostardini',
 'Maria Drapp',
 'Iva Popovic',
 'Dakota Elliott',
 'Silan Demikrol',
 'Anna Kiss',
 'Beste Ayhan',
 'Corrie Brown',
 'Kaylenn Hunt',
 'Megan Depoorter',
 'Bradley',
 'EIU',
 'Reception',
 'Set',
 'Attack',
 'Serve',
 'Block',
 'Dig',
 'Freeball']

In [1092]:
# Sanky graphic step 1 (option2)
# dft = df_v
# segments = dft['player_name'].unique().tolist()
# categories = dft['point_won_by'].unique().tolist()
# subcategories = dft['skill'].unique().tolist()
# labels = segments + subcategories + categories
# labels

In [1093]:
# Sanky graph step 2
sources = []
targets = []
values = []

In [1094]:
# # Sanky graph step 3 (player-skill-win)
for segment in segments:
    for subcat in subcategories:
        dft1 = dft.loc[dft['player_name']==segment]
        dft1 = dft1.loc[dft['point_won_by']==subcat,'point_id'].count()
        sources.append(labels.index(segment))
        targets.append(labels.index(subcat))
        values.append(dft1)
        
for categorie in categories:
    for subcat in dft['point_won_by'].unique():
        dft2 = dft.loc[dft['skill']==categorie]
        dft2 = dft2.loc[dft['point_won_by']==subcat,'point_id'].count()
        sources.append(labels.index(subcat))
        targets.append(labels.index(categorie))
        values.append(dft2)

In [1097]:
# Sanky graph step 4
import plotly.graph_objects as go

fig_sanky_v = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = labels,
      color = "cyan"
    ),
    link = dict(
      source = sources,
      target = targets,
      value = values
  ))])

fig_sanky_v.update_layout(title_text= f"{v_name} relation between Player Skills & Win Rallies", font_size=10)

fig_sanky_v

# 25 Game Summary


In [1098]:
print(f"{win_team} | EIU Match #{g_num} of the {season} season {season_year}")

Bradley win 3-2 vs EIU | EIU Match #7 of the non-conference season 2024


In [483]:
# game_summary_h_3 = print(f"{win_team}\n\n\nAll Serving Rallies: {serve_all_count.sum()}\n{serve_all_count}\n\n{pivot_all_serve_h}\n-----------------------------------\n{set1_serve}\n\nServing rallies Set 1: {set_1_serve_teams.sum()}\n{set_1_serve_teams}\n\nFinal Score Set 1:\n{set_1_serve_points}\n\n{pivot_set1_serve_h}\n\n------------------------------------\n{set2_serve}\n\nServing rallies Set 2: {set_2_serve_teams.sum()}\n{set_2_serve_teams}\n\nFinal Score Set 2:\n{set_2_serve_points}\n\n{pivot_set2_serve_h}\n\n------------------------------------\n{set3_serve}\n\nServing rallies Set 3: {set_3_serve_teams.sum()}\n{set_3_serve_teams}\n\nFinal Score Set 3:\n{set_3_serve_points}\n\n{pivot_set3_serve_h}\n\n------------------------------------\n")
# game_summary_h_3

# 26 CAREFULLLL.... export html
Be CAREFUL with the file`s names

## Chart All points Sequence

In [163]:
# exporting chart point sequence

# fig_line_p.write_html(f'2024_8_30_{h_name}_vs_{v_name}_point_sequence_by_sets.html')

In [165]:
# fig_line_p.write_json(f"2024-8-30-{h_name}_{v_name}-points-seqence.json")

## Check the file names.. Home export SERVE point sequence

In [159]:
h_name

'EIU'

In [162]:
# HOME exporting chart point sequence by rows

# fig_row_serve_h.write_html(f'2024_8_30_{h_name}_serve_rows.html')
# fig_row_rec_h.write_html(f'2024_8_30_{h_name}_rec_rows.html')

## Check the names... VISIT export Serve-Receive sequence by rows

In [160]:
v_name

'Marquette'

In [161]:
# VISIT exporting chart point sequence by rows

# fig_row_serve_v.write_html(f'2024_8_30_{v_name}_serve_rows.html')
# fig_row_rec_v.write_html(f'2024_8_30_{v_name}_rec_rows.html')

In [ ]:
# write the figure to HTML

# 7
# fig_line_p.write_html('2023_11_15_isu_tex_point_sequence_by_sets.html')
# fig_line_row_h.write_html('2023_11_15_isu_tex_home_points_by_setter_pos.html')
# fig_line_row_v.write_html('2023_11_15_isu_tex_visit_points_by_setter_pos.html')

# # 4-1 serve by team
# fig_serve_all.write_html('2023_11_15_isu_tex_serve_by_team.html')

# # 4-2 serve by set
# fig_serve_set.write_html('2023_11_15_isu_tex_serve_by_set.html')

# 4-3 serve by rotation
# fig_serve_v_row.write_html('2023_11_15_isu_tex_serve_isu_by_setter_position.html')

# # 6-1 reception by team
# fig_rec_all.write_html('2023_11_15_isu_tex_receive_isu_by_team.html')

# # 6-2 reception by rotation
# fig_rec_v_row.write_html('2023_11_15_isu_tex_receive_isu_by_setter_position.html')

In [148]:
# example to export multi plot graphs in one HTML file
# fig1.write_html('plots.html', auto_play=True)
# fig2.write_html('plots.html', auto_play=True)
# fig3.write_html('plots.html', auto_play=True)
# fig4.write_html('plots.html', auto_play=True)
# fig5.write_html('plots.html', auto_play=True)
# fig6.write_html('plots.html', auto_play=True)
# fig7.write_html('plots.html', auto_play=True)
# fig8.write_html('plots.html', auto_play=True)
# fig9.write_html('plots.html', auto_play=True)
# fig10.write_html('plots.html', auto_play=True)

# 27 Send to Plotly Dash

In [165]:
# sending the graph to plotly studio
# py.plot(fig_sanky_h, filename = 'Marquette_24nov2023_Sanky_players_skills_win_lost', auto_open=False)

# 98 Advance Stats

## correlation

In [162]:
plt.figure(figsize=(20,16))
sns.heatmap(df.corr(), fmt='.2g', annot=True)

C:\Users\456402\AppData\Local\Temp\ipykernel_7612\2131404039.py:2: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



<Axes: >

# 99 CLOSE THE DATABASE CONNECTION BEFORE CLOSE THE FILE

In [3132]:
# conn.close()